## ETL: Add is_holiday flag to each session

In [2]:
import os
import lxml

import pandas as pd

#### Use maximum width of notebook

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [4]:
!whoami

hduser


In [5]:
os.chdir("/data_data/session_length/")
!pwd

/data_data/session_length


### Make sure no other spark jobs running before doing this

In [6]:
#import pymongo
#import pymongo_spark

#pymongo_spark.activate()


In [7]:
#sc.stop()
#spark.stop()

### If you want to restart Yarn etc

In [8]:
!export HADOOP_HOME=/usr/local/hadoop 
!export PATH=$PATH:/usr/local/hadoop/bin:/usr/local/hadoop/sbin

In [9]:
#!/usr/local/hadoop/sbin/stop-all.sh
#!/usr/local/hadoop/sbin/start-all.sh

In [10]:
APP_NAME = "Add Is_Holiday Flag"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

try:
    sc and spark
except (NameError, UnboundLocalError) as e:
    import findspark
    #findspark.init()
    import pyspark
    import pyspark.sql
    
    #sc = pyspark.SparkContext()
    #spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()
    
sc = SparkContext(conf = SparkConf() .set("spark.sql.autoBroadcastJoinThreshold",-1) .set("spark.driver.maxResultSize", "40g") .set("spark.sql.execution.arrow.enabled", "true") .set('spark.sql.broadcastTimeout', 1000) .set('spark.local.dir', '/data_data/session_length/spark_tmp/') .set('spark.driver.memory', '80G') .set("spark.executor.instances", "20") .set("spark.executor.cores", 20) .set("spark.executor.memory", "12G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")
    

### Check configuration of Spark Environment

In [11]:
sc._conf.getAll()

[(u'spark.local.dir', u'/data_data/session_length/spark_tmp/'),
 (u'spark.sql.execution.arrow.enabled', u'true'),
 (u'spark.executor.cores', u'20'),
 (u'spark.app.id', u'local-1540523624705'),
 (u'spark.driver.memory', u'80G'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.name', u'pyspark-shell'),
 (u'spark.executor.instances', u'20'),
 (u'spark.driver.maxResultSize', u'40g'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.host', u'ip-172-31-41-62.ec2.internal'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.sql.autoBroadcastJoinThreshold', u'-1'),
 (u'spark.sql.broadcastTimeout', u'1000'),
 (u'spark.executor.memory', u'12G'),
 (u'spark.ui.showConsoleProgress', u'true'),
 (u'spark.driver.port', u'37702')]

In [12]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr

from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from datetime import datetime

### Import Data

#### Import utility tools

#### Import both data sets

In [13]:
!head data/lastFM/lastfm-dataset-1K/userid-profile.tsv

#id	gender	age	country	registered
user_000001	m		Japan	Aug 13, 2006
user_000002	f		Peru	Feb 24, 2006
user_000003	m	22	United States	Oct 30, 2005
user_000004	f			Apr 26, 2006
user_000005	m		Bulgaria	Jun 29, 2006
user_000006		24	Russian Federation	May 18, 2006
user_000007	f		United States	Jan 22, 2006
user_000008	m	23	Slovakia	Sep 28, 2006
user_000009	f	19	United States	Jan 13, 2007


In [14]:
users_df = None

In [15]:
from_pattern = 'MMM dd, yyyy'
to_pattern = 'yyyy-MM-dd'

def import_user_data():
    global users_df
    print("==================================================================================================================")
    print("======================================== IMPORTING USERID-PROFILE.CSV  ======================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('gender',  StringType(), True),\
        StructField('age',     IntegerType(), True),\
        StructField('country', StringType(), True),\
        StructField('registered', StringType(), True)])
    users = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-profile.tsv')
    users.show(5,False)
    #func = udf(lambda x: datetime.strptime(x, to_pattern), DateType() )
    users_df = users.withColumn('reg_date', to_date(col("registered"), from_pattern))
    users_df.show(5,False)

In [16]:
import_user_data()

======================================== IMPORTING USERID-PROFILE.CSV  ======================================


+-----------+------+----+-------------+------------+
|userid     |gender|age |country      |registered  |
+-----------+------+----+-------------+------------+
|null       |null  |null|null         |null        |
|user_000001|m     |null|Japan        |Aug 13, 2006|
|user_000002|f     |null|Peru         |Feb 24, 2006|
|user_000003|m     |22  |United States|Oct 30, 2005|
|user_000004|f     |null|null         |Apr 26, 2006|
+-----------+------+----+-------------+------------+
only showing top 5 rows

+-----------+------+----+-------------+------------+----------+
|userid     |gender|age |country      |registered  |reg_date  |
+-----------+------+----+-------------+------------+----------+
|null       |null  |null|null         |null        |null      |
|user_000001|m     |null|Japan        |Aug 13, 2006|2006-08-13|
|user_000002|f     |null|Peru         |Feb 24, 2006|2006-02-24|
|u

#### Import music_data

In [17]:
!head data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv

user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Improvisation (Live_2009_4_15)
user_000001	2009-05-04T13:23:45Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Glacier (Live_2009_4_15)
user_000001	2009-05-04T13:19:22Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Parolibre (Live_2009_4_15)
user_000001	2009-05-04T13:13:38Z	a7f7df4a-77d8-4f12-8acd

In [18]:
sessions_DF = None
sessions_df = None

In [19]:
from_pattern = 'yyyy-MM-ddTHH:mm:ssZ'
to_pattern = 'yyyy-MM-dd'

def import_session_data():
    global sessions_DF
    global sessions_df
    print("====================================================================================================================")
    print("====================================userid-timestamp-artid-artname-traid-traname.tsv================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('timestamp',  StringType(), True),\
        StructField('artid',     StringType(), True),\
        StructField('artname_', StringType(), True),\
        StructField('traid', StringType(), True),\
        StructField('traname_', StringType(), True)
        ])
    sessions = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv')
    sessions.show(5,False)
    #Assign window to time (using hour as window....)
    def get_window(x):
        return int(str(x).split("T")[1].split(":")[0])
    getWindow = udf(lambda timestamp:  get_window(timestamp))
    sessions_DF = sessions.withColumn('session_date',to_date(col('timestamp'))).withColumn('session_window', getWindow(col('timestamp')))
    sessions_DF.show(5,False)
    #Translate artname_ to English
    def translate_to_english(str_to_translate):
        return gs.translate(str_to_translate, 'en')
    func = udf(lambda str_to_translate:  translate_to_english(str_to_translate), StringType())
    #sessions_df = sessions_DF.withColumn('artname', translate_to_english('artname_'))
    #sessions_df[["artname_"]].apply(lambda row: np.mean(row),axis=0)
    #sessions_df["artname"] = sessions_DF["artname_"].apply(lambda name: translate_to_english(name),axis=0)
    #.withColumn('traname', func(col('traname_')))
    sessions_df = sessions_DF.rdd.map(lambda x: (x["artname_"]))
    print(sessions_df.take(5))

In [20]:
import_session_data()

====================================userid-timestamp-artid-artname-traid-traname.tsv================================


+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|userid     |timestamp           |artid                               |artname_ |traid|traname_                                  |
+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e99-8624-24a6e15f133a|Deep Dish|null |Fuck Me Im Famous (Pacha Ibiza)-09-28-2007|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Composition 0919 (Live_2009_4_15)         |
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Mc2 (Live_2009_4_15)                      |
|user_000001|2009-05-04T13:42:52Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Hibari (

In [21]:
#from pyspark.sql.types import Row

#row = Row("artname_")
#df_sessions = sessions_df.map(row).toDF()
#schema = StructType([StructField("artname", StringType(), True)])
#df_sessions = sessions_df.toDF(["artname"],)
#[gs.translate(x,'en') for x in df_sessions]
#df_sessions = sessions_df.flatMap(gs.translate)
#df_sessions.take(5)

In [22]:
users_df.show()

+-----------+------+----+------------------+------------+----------+
|     userid|gender| age|           country|  registered|  reg_date|
+-----------+------+----+------------------+------------+----------+
|       null|  null|null|              null|        null|      null|
|user_000001|     m|null|             Japan|Aug 13, 2006|2006-08-13|
|user_000002|     f|null|              Peru|Feb 24, 2006|2006-02-24|
|user_000003|     m|  22|     United States|Oct 30, 2005|2005-10-30|
|user_000004|     f|null|              null|Apr 26, 2006|2006-04-26|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|2006-06-29|
|user_000006|  null|  24|Russian Federation|May 18, 2006|2006-05-18|
|user_000007|     f|null|     United States|Jan 22, 2006|2006-01-22|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|2006-09-28|
|user_000009|     f|  19|     United States|Jan 13, 2007|2007-01-13|
|user_000010|     m|  19|            Poland| May 4, 2006|2006-05-04|
|user_000011|     m|  21|         

#### Denormalize by joining user and session databases

In [23]:
users_df = users_df.withColumnRenamed('userid','user_id')
u = users_df.alias('u')
s = sessions_DF.alias('s')
join_condition = [ (u.user_id == s.userid) ]
sessionsDF = s.join(u, join_condition, 'inner').drop('user_id')
sessionsDF.show(20,False)

+-----------+--------------------+------------------------------------+------------------+------------------------------------+---------------------------+------------+--------------+------+---+-------------+-----------+----------+
|userid     |timestamp           |artid                               |artname_          |traid                               |traname_                   |session_date|session_window|gender|age|country      |registered |reg_date  |
+-----------+--------------------+------------------------------------+------------------+------------------------------------+---------------------------+------------+--------------+------+---+-------------+-----------+----------+
|user_000066|2009-03-27T17:18:26Z|null                                |The Amen Corner   |null                                |Our Love (Is In The Pocket)|2009-03-27  |17            |f     |20 |United States|May 9, 2006|2006-05-09|
|user_000066|2009-03-08T04:06:21Z|2ceb4e66-4eaa-4dba-ad3a-30df3b742557|N

### Add Genre Data

In [61]:
!head genres_file.csv

","	["bf710b71-48e5-4e15-9bd6-96debb2e4e98", "electro", "swedish", "electronic"]
","	["a94a7155-c79d-4409-9fcf-220cb0e4dc3a", "new wave", "rock", "alternative"]
","	["d41a6875-b626-4c0f-89a1-aecb643d29ff", "irish", "folk", "folk rock"]
","	["5441c29d-3602-4898-b1a1-b77fa23b8e50", "ambient", "instrumental", "70s"]
","	["5dedf5cf-a598-4408-9556-3bf3f149f3ba", "hard rock", "classic rock", "rock"]
","	["36bfa85f-737b-41db-a8fc-b8825850ffc3", "indie", "indie rock", "lo-fi"]
","	["d43d12a1-2dc9-4257-a2fd-0a3bb1081b86", "country", "folk", "johnny cash"]
","	["f41490ce-fe39-435d-86c0-ab5ce098b423", "80s", "new wave", "rock"]
","	["9e53f84d-ef44-4c16-9677-5fd4d78cbd7d", "rock", "alt-country", "alt country"]
","	["d8915e13-d67a-4aa0-9c0b-1f126af951af", "electronic", "british i like", "electronica"]


In [64]:
genres_df = pd.read_csv('genres_file.csv')
genres_df.head()

,", [""bf710b71-48e5-4e15-9bd6-96debb2e4e98""","""electro""","""swedish""","""electronic""]"
0,",\t[""a94a7155-c79d-4409-9fcf-220cb0e4dc3a""","""new wave""","""rock""","""alternative""]"
1,",\t[""d41a6875-b626-4c0f-89a1-aecb643d29ff""","""irish""","""folk""","""folk rock""]"
2,",\t[""5441c29d-3602-4898-b1a1-b77fa23b8e50""","""ambient""","""instrumental""","""70s""]"
3,",\t[""5dedf5cf-a598-4408-9556-3bf3f149f3ba""","""hard rock""","""classic rock""","""rock""]"
4,",\t[""36bfa85f-737b-41db-a8fc-b8825850ffc3""","""indie""","""indie rock""","""lo-fi""]"


#### Save as TSV file for Time Series analysis

#### Uncomment if you want a fresh copy of the file

In [24]:
#!rm -R /data_data/session_length/data/tmp
#sessionsDF.coalesce(1).write.option("delimiter", "\t").csv('/data_data/session_length/data/tmp', header="True")
#!mv /data_data/session_length/data/tmp/part-* /data_data/session_length/data/Session_data.csv

In [25]:
!head /data_data/session_length/data/Session_data.csv

userid	timestamp	artid	artname_	traid	traname_	session_window	gender	age	country	registered	reg_date
user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007	23	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f1

### Get Local time by converting from UTC to Local [WORK IN PROGRESS]

In [26]:
from dateutil import tz
import pytz

### Use countryInfo from here: https://gist.githubusercontent.com/pamelafox/986163/raw/f5f9db4f1b287804fd07ffb3296ed0036292bc7a/countryinfo.py

In [27]:
countries_info = [
{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'], 'code': 'AR', 'continent': 'South America', 'name': 'Argentina', 'capital': 'Buenos Aires'},
{'timezones': ['Europe/Vienna'], 'code': 'AT', 'continent': 'Europe', 'name': 'Austria', 'capital': 'Vienna'},
{'timezones': ['Australia/Lord_Howe', 'Australia/Hobart', 'Australia/Currie', 'Australia/Melbourne', 'Australia/Sydney', 'Australia/Broken_Hill', 'Australia/Brisbane', 'Australia/Lindeman', 'Australia/Adelaide', 'Australia/Darwin', 'Australia/Perth'], 'code': 'AU', 'continent': 'Oceania', 'name': 'Australia', 'capital': 'Canberra'},
{'timezones': ['Asia/Baku'], 'code': 'AZ', 'continent': 'Asia', 'name': 'Azerbaijan', 'capital': 'Baku'},
{'timezones': ['America/Barbados'], 'code': 'BB', 'continent': 'North America', 'name': 'Barbados', 'capital': 'Bridgetown'},
{'timezones': ['Asia/Dhaka'], 'code': 'BD', 'continent': 'Asia', 'name': 'Bangladesh', 'capital': 'Dhaka'},
{'timezones': ['Europe/Brussels'], 'code': 'BE', 'continent': 'Europe', 'name': 'Belgium', 'capital': 'Brussels'},
{'timezones': ['Africa/Ouagadougou'], 'code': 'BF', 'continent': 'Africa', 'name': 'Burkina Faso', 'capital': 'Ouagadougou'},
{'timezones': ['Europe/Sofia'], 'code': 'BG', 'continent': 'Europe', 'name': 'Bulgaria', 'capital': 'Sofia'},
{'timezones': ['Asia/Bahrain'], 'code': 'BH', 'continent': 'Asia', 'name': 'Bahrain', 'capital': 'Manama'},
{'timezones': ['Africa/Bujumbura'], 'code': 'BI', 'continent': 'Africa', 'name': 'Burundi', 'capital': 'Bujumbura'},
{'timezones': ['Africa/Porto-Novo'], 'code': 'BJ', 'continent': 'Africa', 'name': 'Benin', 'capital': 'Porto-Novo'},
{'timezones': ['Asia/Brunei'], 'code': 'BN', 'continent': 'Asia', 'name': 'Brunei Darussalam', 'capital': 'Bandar Seri Begawan'},
{'timezones': ['America/La_Paz'], 'code': 'BO', 'continent': 'South America', 'name': 'Bolivia', 'capital': 'Sucre'},
{'timezones': ['America/Noronha', 'America/Belem', 'America/Fortaleza', 'America/Recife', 'America/Araguaina', 'America/Maceio', 'America/Bahia', 'America/Sao_Paulo', 'America/Campo_Grande', 'America/Cuiaba', 'America/Porto_Velho', 'America/Boa_Vista', 'America/Manaus', 'America/Eirunepe', 'America/Rio_Branco'], 'code': 'BR', 'continent': 'South America', 'name': 'Brazil', 'capital': 'Bras\xc3\xadlia'},
{'timezones': ['America/Nassau'], 'code': 'BS', 'continent': 'North America', 'name': 'Bahamas', 'capital': 'Nassau'},
{'timezones': ['Asia/Thimphu'], 'code': 'BT', 'continent': 'Asia', 'name': 'Bhutan', 'capital': 'Thimphu'},
{'timezones': ['Africa/Gaborone'], 'code': 'BW', 'continent': 'Africa', 'name': 'Botswana', 'capital': 'Gaborone'},
{'timezones': ['Europe/Minsk'], 'code': 'BY', 'continent': 'Europe', 'name': 'Belarus', 'capital': 'Minsk'},
{'timezones': ['America/Belize'], 'code': 'BZ', 'continent': 'North America', 'name': 'Belize', 'capital': 'Belmopan'},
{'timezones': ['America/St_Johns', 'America/Halifax', 'America/Glace_Bay', 'America/Moncton', 'America/Goose_Bay', 'America/Blanc-Sablon', 'America/Montreal', 'America/Toronto', 'America/Nipigon', 'America/Thunder_Bay', 'America/Pangnirtung', 'America/Iqaluit', 'America/Atikokan', 'America/Rankin_Inlet', 'America/Winnipeg', 'America/Rainy_River', 'America/Cambridge_Bay', 'America/Regina', 'America/Swift_Current', 'America/Edmonton', 'America/Yellowknife', 'America/Inuvik', 'America/Dawson_Creek', 'America/Vancouver', 'America/Whitehorse', 'America/Dawson'], 'code': 'CA', 'continent': 'North America', 'name': 'Canada', 'capital': 'Ottawa'},
{'timezones': ['Africa/Kinshasa', 'Africa/Lubumbashi'], 'code': 'CD', 'continent': 'Africa', 'name': 'Democratic Republic of the Congo', 'capital': 'Kinshasa'},
{'timezones': ['Africa/Brazzaville'], 'code': 'CG', 'continent': 'Africa', 'name': 'Republic of the Congo', 'capital': 'Brazzaville'},
{'timezones': ['Africa/Abidjan'], 'code': 'CI', 'continent': 'Africa', 'name': "C\xc3\xb4te d'Ivoire", 'capital': 'Yamoussoukro'},
{'timezones': ['America/Santiago', 'Pacific/Easter'], 'code': 'CL', 'continent': 'South America', 'name': 'Chile', 'capital': 'Santiago'},
{'timezones': ['Africa/Douala'], 'code': 'CM', 'continent': 'Africa', 'name': 'Cameroon', 'capital': 'Yaound\xc3\xa9'},
{'timezones': ['Asia/Shanghai', 'Asia/Harbin', 'Asia/Chongqing', 'Asia/Urumqi', 'Asia/Kashgar'], 'code': 'CN', 'continent': 'Asia', 'name': "People's Republic of China", 'capital': 'Beijing'},
{'timezones': ['America/Bogota'], 'code': 'CO', 'continent': 'South America', 'name': 'Colombia', 'capital': 'Bogot\xc3\xa1'},
{'timezones': ['America/Costa_Rica'], 'code': 'CR', 'continent': 'North America', 'name': 'Costa Rica', 'capital': 'San Jos\xc3\xa9'},
{'timezones': ['America/Havana'], 'code': 'CU', 'continent': 'North America', 'name': 'Cuba', 'capital': 'Havana'},
{'timezones': ['Atlantic/Cape_Verde'], 'code': 'CV', 'continent': 'Africa', 'name': 'Cape Verde', 'capital': 'Praia'},
{'timezones': ['Asia/Nicosia'], 'code': 'CY', 'continent': 'Asia', 'name': 'Cyprus', 'capital': 'Nicosia'},
{'timezones': ['Europe/Prague'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czech Republic', 'capital': 'Prague'},
{'timezones': ['Europe/Berlin'], 'code': 'DE', 'continent': 'Europe', 'name': 'Germany', 'capital': 'Berlin'},
{'timezones': ['Africa/Djibouti'], 'code': 'DJ', 'continent': 'Africa', 'name': 'Djibouti', 'capital': 'Djibouti City'},
{'timezones': ['Europe/Copenhagen'], 'code': 'DK', 'continent': 'Europe', 'name': 'Denmark', 'capital': 'Copenhagen'},
{'timezones': ['America/Dominica'], 'code': 'DM', 'continent': 'North America', 'name': 'Dominica', 'capital': 'Roseau'},
{'timezones': ['America/Santo_Domingo'], 'code': 'DO', 'continent': 'North America', 'name': 'Dominican Republic', 'capital': 'Santo Domingo'},
{'timezones': ['America/Guayaquil', 'Pacific/Galapagos'], 'code': 'EC', 'continent': 'South America', 'name': 'Ecuador', 'capital': 'Quito'},
{'timezones': ['Europe/Tallinn'], 'code': 'EE', 'continent': 'Europe', 'name': 'Estonia', 'capital': 'Tallinn'},
{'timezones': ['Africa/Cairo'], 'code': 'EG', 'continent': 'Africa', 'name': 'Egypt', 'capital': 'Cairo'},
{'timezones': ['Africa/Asmera'], 'code': 'ER', 'continent': 'Africa', 'name': 'Eritrea', 'capital': 'Asmara'},
{'timezones': ['Africa/Addis_Ababa'], 'code': 'ET', 'continent': 'Africa', 'name': 'Ethiopia', 'capital': 'Addis Ababa'},
{'timezones': ['Europe/Helsinki'], 'code': 'FI', 'continent': 'Europe', 'name': 'Finland', 'capital': 'Helsinki'},
{'timezones': ['Pacific/Fiji'], 'code': 'FJ', 'continent': 'Oceania', 'name': 'Fiji', 'capital': 'Suva'},
{'timezones': ['Europe/Paris'], 'code': 'FR', 'continent': 'Europe', 'name': 'France', 'capital': 'Paris'},
{'timezones': ['Africa/Libreville'], 'code': 'GA', 'continent': 'Africa', 'name': 'Gabon', 'capital': 'Libreville'},
{'timezones': ['Asia/Tbilisi'], 'code': 'GE', 'continent': 'Asia', 'name': 'Georgia', 'capital': 'Tbilisi'},
{'timezones': ['Africa/Accra'], 'code': 'GH', 'continent': 'Africa', 'name': 'Ghana', 'capital': 'Accra'},
{'timezones': ['Africa/Banjul'], 'code': 'GM', 'continent': 'Africa', 'name': 'The Gambia', 'capital': 'Banjul'},
{'timezones': ['Africa/Conakry'], 'code': 'GN', 'continent': 'Africa', 'name': 'Guinea', 'capital': 'Conakry'},
{'timezones': ['Europe/Athens'], 'code': 'GR', 'continent': 'Europe', 'name': 'Greece', 'capital': 'Athens'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Guatemala', 'capital': 'Guatemala City'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Haiti', 'capital': 'Port-au-Prince'},
{'timezones': ['Africa/Bissau'], 'code': 'GW', 'continent': 'Africa', 'name': 'Guinea-Bissau', 'capital': 'Bissau'},
{'timezones': ['America/Guyana'], 'code': 'GY', 'continent': 'South America', 'name': 'Guyana', 'capital': 'Georgetown'},
{'timezones': ['America/Tegucigalpa'], 'code': 'HN', 'continent': 'North America', 'name': 'Honduras', 'capital': 'Tegucigalpa'},
{'timezones': ['Europe/Budapest'], 'code': 'HU', 'continent': 'Europe', 'name': 'Hungary', 'capital': 'Budapest'},
{'timezones': ['Asia/Jakarta', 'Asia/Pontianak', 'Asia/Makassar', 'Asia/Jayapura'], 'code': 'ID', 'continent': 'Asia', 'name': 'Indonesia', 'capital': 'Jakarta'},
{'timezones': ['Europe/Dublin'], 'code': 'IE', 'continent': 'Europe', 'name': 'Republic of Ireland', 'capital': 'Dublin'},
{'timezones': ['Asia/Jerusalem'], 'code': 'IL', 'continent': 'Asia', 'name': 'Israel', 'capital': 'Jerusalem'},
{'timezones': ['Asia/Calcutta'], 'code': 'IN', 'continent': 'Asia', 'name': 'India', 'capital': 'New Delhi'},
{'timezones': ['Asia/Baghdad'], 'code': 'IQ', 'continent': 'Asia', 'name': 'Iraq', 'capital': 'Baghdad'},
{'timezones': ['Asia/Tehran'], 'code': 'IR', 'continent': 'Asia', 'name': 'Iran', 'capital': 'Tehran'},
{'timezones': ['Atlantic/Reykjavik'], 'code': 'IS', 'continent': 'Europe', 'name': 'Iceland', 'capital': 'Reykjav\xc3\xadk'},
{'timezones': ['Europe/Rome'], 'code': 'IT', 'continent': 'Europe', 'name': 'Italy', 'capital': 'Rome'},
{'timezones': ['America/Jamaica'], 'code': 'JM', 'continent': 'North America', 'name': 'Jamaica', 'capital': 'Kingston'},
{'timezones': ['Asia/Amman'], 'code': 'JO', 'continent': 'Asia', 'name': 'Jordan', 'capital': 'Amman'},
{'timezones': ['Asia/Tokyo'], 'code': 'JP', 'continent': 'Asia', 'name': 'Japan', 'capital': 'Tokyo'},
{'timezones': ['Africa/Nairobi'], 'code': 'KE', 'continent': 'Africa', 'name': 'Kenya', 'capital': 'Nairobi'},
{'timezones': ['Asia/Bishkek'], 'code': 'KG', 'continent': 'Asia', 'name': 'Kyrgyzstan', 'capital': 'Bishkek'},
{'timezones': ['Pacific/Tarawa', 'Pacific/Enderbury', 'Pacific/Kiritimati'], 'code': 'KI', 'continent': 'Oceania', 'name': 'Kiribati', 'capital': 'Tarawa'},
{'timezones': ['Asia/Pyongyang'], 'code': 'KP', 'continent': 'Asia', 'name': 'North Korea', 'capital': 'Pyongyang'},
{'timezones': ['Asia/Seoul'], 'code': 'KR', 'continent': 'Asia', 'name': 'South Korea', 'capital': 'Seoul'},
{'timezones': ['Asia/Kuwait'], 'code': 'KW', 'continent': 'Asia', 'name': 'Kuwait', 'capital': 'Kuwait City'},
{'timezones': ['Asia/Beirut'], 'code': 'LB', 'continent': 'Asia', 'name': 'Lebanon', 'capital': 'Beirut'},
{'timezones': ['Europe/Vaduz'], 'code': 'LI', 'continent': 'Europe', 'name': 'Liechtenstein', 'capital': 'Vaduz'},
{'timezones': ['Africa/Monrovia'], 'code': 'LR', 'continent': 'Africa', 'name': 'Liberia', 'capital': 'Monrovia'},
{'timezones': ['Africa/Maseru'], 'code': 'LS', 'continent': 'Africa', 'name': 'Lesotho', 'capital': 'Maseru'},
{'timezones': ['Europe/Vilnius'], 'code': 'LT', 'continent': 'Europe', 'name': 'Lithuania', 'capital': 'Vilnius'},
{'timezones': ['Europe/Luxembourg'], 'code': 'LU', 'continent': 'Europe', 'name': 'Luxembourg', 'capital': 'Luxembourg City'},
{'timezones': ['Europe/Riga'], 'code': 'LV', 'continent': 'Europe', 'name': 'Latvia', 'capital': 'Riga'},
{'timezones': ['Africa/Tripoli'], 'code': 'LY', 'continent': 'Africa', 'name': 'Libya', 'capital': 'Tripoli'},
{'timezones': ['Indian/Antananarivo'], 'code': 'MG', 'continent': 'Africa', 'name': 'Madagascar', 'capital': 'Antananarivo'},
{'timezones': ['Pacific/Majuro', 'Pacific/Kwajalein'], 'code': 'MH', 'continent': 'Oceania', 'name': 'Marshall Islands', 'capital': 'Majuro'},
{'timezones': ['Europe/Skopje'], 'code': 'MK', 'continent': 'Europe', 'name': 'Macedonia', 'capital': 'Skopje'},
{'timezones': ['Africa/Bamako'], 'code': 'ML', 'continent': 'Africa', 'name': 'Mali', 'capital': 'Bamako'},
{'timezones': ['Asia/Rangoon'], 'code': 'MM', 'continent': 'Asia', 'name': 'Myanmar', 'capital': 'Naypyidaw'},
{'timezones': ['Asia/Ulaanbaatar', 'Asia/Hovd', 'Asia/Choibalsan'], 'code': 'MN', 'continent': 'Asia', 'name': 'Mongolia', 'capital': 'Ulaanbaatar'},
{'timezones': ['Africa/Nouakchott'], 'code': 'MR', 'continent': 'Africa', 'name': 'Mauritania', 'capital': 'Nouakchott'},
{'timezones': ['Europe/Malta'], 'code': 'MT', 'continent': 'Europe', 'name': 'Malta', 'capital': 'Valletta'},
{'timezones': ['Indian/Mauritius'], 'code': 'MU', 'continent': 'Africa', 'name': 'Mauritius', 'capital': 'Port Louis'},
{'timezones': ['Indian/Maldives'], 'code': 'MV', 'continent': 'Asia', 'name': 'Maldives', 'capital': 'Mal\xc3\xa9'},
{'timezones': ['Africa/Blantyre'], 'code': 'MW', 'continent': 'Africa', 'name': 'Malawi', 'capital': 'Lilongwe'},
{'timezones': ['America/Mexico_City', 'America/Cancun', 'America/Merida', 'America/Monterrey', 'America/Mazatlan', 'America/Chihuahua', 'America/Hermosillo', 'America/Tijuana'], 'code': 'MX', 'continent': 'North America', 'name': 'Mexico', 'capital': 'Mexico City'},
{'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching'], 'code': 'MY', 'continent': 'Asia', 'name': 'Malaysia', 'capital': 'Kuala Lumpur'},
{'timezones': ['Africa/Maputo'], 'code': 'MZ', 'continent': 'Africa', 'name': 'Mozambique', 'capital': 'Maputo'},
{'timezones': ['Africa/Windhoek'], 'code': 'NA', 'continent': 'Africa', 'name': 'Namibia', 'capital': 'Windhoek'},
{'timezones': ['Africa/Niamey'], 'code': 'NE', 'continent': 'Africa', 'name': 'Niger', 'capital': 'Niamey'},
{'timezones': ['Africa/Lagos'], 'code': 'NG', 'continent': 'Africa', 'name': 'Nigeria', 'capital': 'Abuja'},
{'timezones': ['America/Managua'], 'code': 'NI', 'continent': 'North America', 'name': 'Nicaragua', 'capital': 'Managua'},
{'timezones': ['Europe/Amsterdam'], 'code': 'NL', 'continent': 'Europe', 'name': 'Kingdom of the Netherlands', 'capital': 'Amsterdam'},
{'timezones': ['Europe/Oslo'], 'code': 'NO', 'continent': 'Europe', 'name': 'Norway', 'capital': 'Oslo'},
{'timezones': ['Asia/Katmandu'], 'code': 'NP', 'continent': 'Asia', 'name': 'Nepal', 'capital': 'Kathmandu'},
{'timezones': ['Pacific/Nauru'], 'code': 'NR', 'continent': 'Oceania', 'name': 'Nauru', 'capital': 'Yaren'},
{'timezones': ['Pacific/Auckland', 'Pacific/Chatham'], 'code': 'NZ', 'continent': 'Oceania', 'name': 'New Zealand', 'capital': 'Wellington'},
{'timezones': ['Asia/Muscat'], 'code': 'OM', 'continent': 'Asia', 'name': 'Oman', 'capital': 'Muscat'},
{'timezones': ['America/Panama'], 'code': 'PA', 'continent': 'North America', 'name': 'Panama', 'capital': 'Panama City'},
{'timezones': ['America/Lima'], 'code': 'PE', 'continent': 'South America', 'name': 'Peru', 'capital': 'Lima'},
{'timezones': ['Pacific/Port_Moresby'], 'code': 'PG', 'continent': 'Oceania', 'name': 'Papua New Guinea', 'capital': 'Port Moresby'},
{'timezones': ['Asia/Manila'], 'code': 'PH', 'continent': 'Asia', 'name': 'Philippines', 'capital': 'Manila'},
{'timezones': ['Asia/Karachi'], 'code': 'PK', 'continent': 'Asia', 'name': 'Pakistan', 'capital': 'Islamabad'},
{'timezones': ['Europe/Warsaw'], 'code': 'PL', 'continent': 'Europe', 'name': 'Poland', 'capital': 'Warsaw'},
{'timezones': ['Europe/Lisbon', 'Atlantic/Madeira', 'Atlantic/Azores'], 'code': 'PT', 'continent': 'Europe', 'name': 'Portugal', 'capital': 'Lisbon'},
{'timezones': ['Pacific/Palau'], 'code': 'PW', 'continent': 'Oceania', 'name': 'Palau', 'capital': 'Ngerulmud'},
{'timezones': ['America/Asuncion'], 'code': 'PY', 'continent': 'South America', 'name': 'Paraguay', 'capital': 'Asunci\xc3\xb3n'},
{'timezones': ['Asia/Qatar'], 'code': 'QA', 'continent': 'Asia', 'name': 'Qatar', 'capital': 'Doha'},
{'timezones': ['Europe/Bucharest'], 'code': 'RO', 'continent': 'Europe', 'name': 'Romania', 'capital': 'Bucharest'},
{'timezones': ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Volgograd', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Yakutsk', 'Asia/Vladivostok', 'Asia/Sakhalin', 'Asia/Magadan', 'Asia/Kamchatka', 'Asia/Anadyr'], 'code': 'RU', 'continent': 'Europe', 'name': 'Russia', 'capital': 'Moscow'},
{'timezones': ['Africa/Kigali'], 'code': 'RW', 'continent': 'Africa', 'name': 'Rwanda', 'capital': 'Kigali'},
{'timezones': ['Asia/Riyadh'], 'code': 'SA', 'continent': 'Asia', 'name': 'Saudi Arabia', 'capital': 'Riyadh'},
{'timezones': ['Pacific/Guadalcanal'], 'code': 'SB', 'continent': 'Oceania', 'name': 'Solomon Islands', 'capital': 'Honiara'},
{'timezones': ['Indian/Mahe'], 'code': 'SC', 'continent': 'Africa', 'name': 'Seychelles', 'capital': 'Victoria'},
{'timezones': ['Africa/Khartoum'], 'code': 'SD', 'continent': 'Africa', 'name': 'Sudan', 'capital': 'Khartoum'},
{'timezones': ['Europe/Stockholm'], 'code': 'SE', 'continent': 'Europe', 'name': 'Sweden', 'capital': 'Stockholm'},
{'timezones': ['Asia/Singapore'], 'code': 'SG', 'continent': 'Asia', 'name': 'Singapore', 'capital': 'Singapore'},
{'timezones': ['Europe/Ljubljana'], 'code': 'SI', 'continent': 'Europe', 'name': 'Slovenia', 'capital': 'Ljubljana'},
{'timezones': ['Europe/Bratislava'], 'code': 'SK', 'continent': 'Europe', 'name': 'Slovakia', 'capital': 'Bratislava'},
{'timezones': ['Africa/Freetown'], 'code': 'SL', 'continent': 'Africa', 'name': 'Sierra Leone', 'capital': 'Freetown'},
{'timezones': ['Europe/San_Marino'], 'code': 'SM', 'continent': 'Europe', 'name': 'San Marino', 'capital': 'San Marino'},
{'timezones': ['Africa/Dakar'], 'code': 'SN', 'continent': 'Africa', 'name': 'Senegal', 'capital': 'Dakar'},
{'timezones': ['Africa/Mogadishu'], 'code': 'SO', 'continent': 'Africa', 'name': 'Somalia', 'capital': 'Mogadishu'},
{'timezones': ['America/Paramaribo'], 'code': 'SR', 'continent': 'South America', 'name': 'Suriname', 'capital': 'Paramaribo'},
{'timezones': ['Africa/Sao_Tome'], 'code': 'ST', 'continent': 'Africa', 'name': 'S\xc3\xa3o Tom\xc3\xa9 and Pr\xc3\xadncipe', 'capital': 'S\xc3\xa3o Tom\xc3\xa9'},
{'timezones': ['Asia/Damascus'], 'code': 'SY', 'continent': 'Asia', 'name': 'Syria', 'capital': 'Damascus'},
{'timezones': ['Africa/Lome'], 'code': 'TG', 'continent': 'Africa', 'name': 'Togo', 'capital': 'Lom\xc3\xa9'},
{'timezones': ['Asia/Bangkok'], 'code': 'TH', 'continent': 'Asia', 'name': 'Thailand', 'capital': 'Bangkok'},
{'timezones': ['Asia/Dushanbe'], 'code': 'TJ', 'continent': 'Asia', 'name': 'Tajikistan', 'capital': 'Dushanbe'},
{'timezones': ['Asia/Ashgabat'], 'code': 'TM', 'continent': 'Asia', 'name': 'Turkmenistan', 'capital': 'Ashgabat'},
{'timezones': ['Africa/Tunis'], 'code': 'TN', 'continent': 'Africa', 'name': 'Tunisia', 'capital': 'Tunis'},
{'timezones': ['Pacific/Tongatapu'], 'code': 'TO', 'continent': 'Oceania', 'name': 'Tonga', 'capital': 'Nuku\xca\xbbalofa'},
{'timezones': ['Europe/Istanbul'], 'code': 'TR', 'continent': 'Asia', 'name': 'Turkey', 'capital': 'Ankara'},
{'timezones': ['America/Port_of_Spain'], 'code': 'TT', 'continent': 'North America', 'name': 'Trinidad and Tobago', 'capital': 'Port of Spain'},
{'timezones': ['Pacific/Funafuti'], 'code': 'TV', 'continent': 'Oceania', 'name': 'Tuvalu', 'capital': 'Funafuti'},
{'timezones': ['Africa/Dar_es_Salaam'], 'code': 'TZ', 'continent': 'Africa', 'name': 'Tanzania', 'capital': 'Dodoma'},
{'timezones': ['Europe/Kiev', 'Europe/Uzhgorod', 'Europe/Zaporozhye', 'Europe/Simferopol'], 'code': 'UA', 'continent': 'Europe', 'name': 'Ukraine', 'capital': 'Kiev'},
{'timezones': ['Africa/Kampala'], 'code': 'UG', 'continent': 'Africa', 'name': 'Uganda', 'capital': 'Kampala'},
{'timezones': ['America/New_York', 'America/Detroit', 'America/Kentucky/Louisville', 'America/Kentucky/Monticello', 'America/Indiana/Indianapolis', 'America/Indiana/Marengo', 'America/Indiana/Knox', 'America/Indiana/Vevay', 'America/Chicago', 'America/Indiana/Vincennes', 'America/Indiana/Petersburg', 'America/Menominee', 'America/North_Dakota/Center', 'America/North_Dakota/New_Salem', 'America/Denver', 'America/Boise', 'America/Shiprock', 'America/Phoenix', 'America/Los_Angeles', 'America/Anchorage', 'America/Juneau', 'America/Yakutat', 'America/Nome', 'America/Adak', 'Pacific/Honolulu'], 'code': 'US', 'continent': 'North America', 'name': 'United States', 'capital': 'Washington, D.C.'},
{'timezones': ['America/Montevideo'], 'code': 'UY', 'continent': 'South America', 'name': 'Uruguay', 'capital': 'Montevideo'},
{'timezones': ['Asia/Samarkand', 'Asia/Tashkent'], 'code': 'UZ', 'continent': 'Asia', 'name': 'Uzbekistan', 'capital': 'Tashkent'},
{'timezones': ['Europe/Vatican'], 'code': 'VA', 'continent': 'Europe', 'name': 'Vatican City', 'capital': 'Vatican City'},
{'timezones': ['America/Caracas'], 'code': 'VE', 'continent': 'South America', 'name': 'Venezuela', 'capital': 'Caracas'},
{'timezones': ['Asia/Saigon'], 'code': 'VN', 'continent': 'Asia', 'name': 'Vietnam', 'capital': 'Hanoi'},
{'timezones': ['Pacific/Efate'], 'code': 'VU', 'continent': 'Oceania', 'name': 'Vanuatu', 'capital': 'Port Vila'},
{'timezones': ['Asia/Aden'], 'code': 'YE', 'continent': 'Asia', 'name': 'Yemen', 'capital': "Sana'a"},
{'timezones': ['Africa/Lusaka'], 'code': 'ZM', 'continent': 'Africa', 'name': 'Zambia', 'capital': 'Lusaka'},
{'timezones': ['Africa/Harare'], 'code': 'ZW', 'continent': 'Africa', 'name': 'Zimbabwe', 'capital': 'Harare'},
{'timezones': ['Africa/Algiers'], 'code': 'DZ', 'continent': 'Africa', 'name': 'Algeria', 'capital': 'Algiers'},
{'timezones': ['Europe/Sarajevo'], 'code': 'BA', 'continent': 'Europe', 'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo'},
{'timezones': ['Asia/Phnom_Penh'], 'code': 'KH', 'continent': 'Asia', 'name': 'Cambodia', 'capital': 'Phnom Penh'},
{'timezones': ['Africa/Bangui'], 'code': 'CF', 'continent': 'Africa', 'name': 'Central African Republic', 'capital': 'Bangui'},
{'timezones': ['Africa/Ndjamena'], 'code': 'TD', 'continent': 'Africa', 'name': 'Chad', 'capital': "N'Djamena"},
{'timezones': ['Indian/Comoro'], 'code': 'KM', 'continent': 'Africa', 'name': 'Comoros', 'capital': 'Moroni'},
{'timezones': ['Europe/Zagreb'], 'code': 'HR', 'continent': 'Europe', 'name': 'Croatia', 'capital': 'Zagreb'},
{'timezones': ['Asia/Dili'], 'code': 'TL', 'continent': 'Asia', 'name': 'East Timor', 'capital': 'Dili'},
{'timezones': ['America/El_Salvador'], 'code': 'SV', 'continent': 'North America', 'name': 'El Salvador', 'capital': 'San Salvador'},
{'timezones': ['Africa/Malabo'], 'code': 'GQ', 'continent': 'Africa', 'name': 'Equatorial Guinea', 'capital': 'Malabo'},
{'timezones': ['America/Grenada'], 'code': 'GD', 'continent': 'North America', 'name': 'Grenada', 'capital': "St. George's"},
{'timezones': ['Asia/Almaty', 'Asia/Qyzylorda', 'Asia/Aqtobe', 'Asia/Aqtau', 'Asia/Oral'], 'code': 'KZ', 'continent': 'Asia', 'name': 'Kazakhstan', 'capital': 'Astana'},
{'timezones': ['Asia/Vientiane'], 'code': 'LA', 'continent': 'Asia', 'name': 'Laos', 'capital': 'Vientiane'},
{'timezones': ['Pacific/Truk', 'Pacific/Ponape', 'Pacific/Kosrae'], 'code': 'FM', 'continent': 'Oceania', 'name': 'Federated States of Micronesia', 'capital': 'Palikir'},
{'timezones': ['Europe/Chisinau'], 'code': 'MD', 'continent': 'Europe', 'name': 'Moldova', 'capital': 'Chi\xc5\x9fin\xc4\x83u'},
{'timezones': ['Europe/Monaco'], 'code': 'MC', 'continent': 'Europe', 'name': 'Monaco', 'capital': 'Monaco'},
{'timezones': ['Europe/Podgorica'], 'code': 'ME', 'continent': 'Europe', 'name': 'Montenegro', 'capital': 'Podgorica'},
{'timezones': ['Africa/Casablanca'], 'code': 'MA', 'continent': 'Africa', 'name': 'Morocco', 'capital': 'Rabat'},
{'timezones': ['America/St_Kitts'], 'code': 'KN', 'continent': 'North America', 'name': 'Saint Kitts and Nevis', 'capital': 'Basseterre'},
{'timezones': ['America/St_Lucia'], 'code': 'LC', 'continent': 'North America', 'name': 'Saint Lucia', 'capital': 'Castries'},
{'timezones': ['America/St_Vincent'], 'code': 'VC', 'continent': 'North America', 'name': 'Saint Vincent and the Grenadines', 'capital': 'Kingstown'},
{'timezones': ['Pacific/Apia'], 'code': 'WS', 'continent': 'Oceania', 'name': 'Samoa', 'capital': 'Apia'},
{'timezones': ['Europe/Belgrade'], 'code': 'RS', 'continent': 'Europe', 'name': 'Serbia', 'capital': 'Belgrade'},
{'timezones': ['Africa/Johannesburg'], 'code': 'ZA', 'continent': 'Africa', 'name': 'South Africa', 'capital': 'Pretoria'},
{'timezones': ['Europe/Madrid', 'Africa/Ceuta', 'Atlantic/Canary'], 'code': 'ES', 'continent': 'Europe', 'name': 'Spain', 'capital': 'Madrid'},
{'timezones': ['Asia/Colombo'], 'code': 'LK', 'continent': 'Asia', 'name': 'Sri Lanka', 'capital': 'Sri Jayewardenepura Kotte'},
{'timezones': ['Africa/Mbabane'], 'code': 'SZ', 'continent': 'Africa', 'name': 'Swaziland', 'capital': 'Mbabane'},
{'timezones': ['Europe/Zurich'], 'code': 'CH', 'continent': 'Europe', 'name': 'Switzerland', 'capital': 'Bern'},
{'timezones': ['Asia/Dubai'], 'code': 'AE', 'continent': 'Asia', 'name': 'United Arab Emirates', 'capital': 'Abu Dhabi'},
{'timezones': ['Europe/London'], 'code': 'GB', 'continent': 'Europe', 'name': 'United Kingdom', 'capital': 'London'},
]

In [28]:
country_capital_city_tz = dict()
[country_capital_city_tz.update({name['name']:name['continent'].split()[-1]+"/"+"_".join(name['capital'].split(" ")).replace(",","")}) for name in countries_info]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [29]:
country_capital_city_tz['France']

'Europe/Paris'

In [30]:
country_capital_city_tz['United States']

'America/Washington_D.C.'

In [31]:
import json
with open('country_capital_city_tz.json', 'w') as file:
     file.write(json.dumps(country_capital_city_tz))

In [32]:
!head country_capital_city_tz.json


{"Canada": "America/Ottawa", "East Timor": "Asia/Dili", "Turkmenistan": "Asia/Ashgabat", "Lithuania": "Europe/Vilnius", "Cambodia": "Asia/Phnom_Penh", "Saint Kitts and Nevis": "America/Basseterre", "Ethiopia": "Africa/Addis_Ababa", "The Gambia": "Africa/Banjul", "Sri Lanka": "Asia/Sri_Jayewardenepura_Kotte", "Swaziland": "Africa/Mbabane", "Argentina": "America/Buenos_Aires", "Bolivia": "America/Sucre", "Cameroon": "Africa/Yaound\u00e9", "Burkina Faso": "Africa/Ouagadougou", "Ghana": "Africa/Accra", "Saudi Arabia": "Asia/Riyadh", "Laos": "Asia/Vientiane", "Japan": "Asia/Tokyo", "Republic of Ireland": "Europe/Dublin", "Slovenia": "Europe/Ljubljana", "Guatemala": "America/Guatemala_City", "Bosnia and Herzegovina": "Europe/Sarajevo", "Kuwait": "Asia/Kuwait_City", "Germany": "Europe/Berlin", "Dominica": "America/Roseau", "Liberia": "Africa/Monrovia", "Maldives": "Asia/Mal\u00e9", "Pakistan": "Asia/Islamabad", "Oman": "Asia/Muscat", "Tanzania": "Africa/Dodoma", "Seychelles": "Africa/Victoria

### Add TimeZone information for each session )

### SWITCHING TO Map Reduce

In [33]:
import ast
str_ = "'\\xe5\\x9d\\x82\\xe6\\x9c\\xac\\xe9\\xbe\\x8d\\xe4\\xb8\\x80'"
str_[1:-1]

'\\xe5\\x9d\\x82\\xe6\\x9c\\xac\\xe9\\xbe\\x8d\\xe4\\xb8\\x80'

In [34]:
%%writefile MRTranslate.py

import os

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob import step
from mrjob.protocol import RawProtocol

from collections import defaultdict, Counter

import re
import sys
import json

import datetime
from datetime import datetime
from dateutil import parser
from dateutil import tz
import pytz

import ast

#import goslate 
#import translate
from googletrans import Translator

import requests


class MRTranslate(MRJob):

    def __init__(self, *args, **kwargs):
        super(MRTranslate, self).__init__(*args, **kwargs)
        self.min_line_length = 12
        self.translator   = Translator()
        self.day_of_week  = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
        self.time_of_day  = {"Night":[0,1,2,3,4], "Dawn":[5,6],"Morning":[7,8,9],"LateMorning":[10,11], "Lunch":[12,13],"Afternoon":[14,15,16],"LateAfternoon":[17,18],"Evening":[19,20,21],"LateEvening":[22,23]}

    
    #def configure_options(self):
    #    super(MRTranslate, self).configure_options()
    #    self.add_file_option('--tzFile', default='/user/w210/data/country_capital_city_tz.json', dest='tzFile')
    #    self.add_passthrough_option('--num-mappers',  default=10, type="int", dest="num_mappers",  help="Number of mappers")
    #    self.add_passthrough_option('--num-reducers', default=10, type="int", dest="num_reducers", help="Number of reducers")
    #    self.add_passthru_arg('--local_dir',    default='/user/hduser',  dest="local_dir",  help="Working directory for user")
    #    self.add_passthru_arg('--tmp_dir',      default='/user/hduser',  dest="tmp_dir",    help="Temp directory for user")                                    
        
    def configure_args(self):
        super(MRTranslate, self).configure_args()
        self.add_file_arg('--tzFile', default='/user/w210/data/country_capital_city_tz.json', dest='tzFile')
        self.add_passthru_arg('--num_mappers',  default=10,  dest="num_mappers",  help="Number of mappers")
        self.add_passthru_arg('--num_reducers', default=10,  dest="num_reducers", help="Number of reducers")
        self.add_passthru_arg('--local_dir',    default='/user/hduser',  dest="local_dir",  help="Working directory for user")
        self.add_passthru_arg('--tmp_dir',      default='/user/hduser',  dest="tmp_dir",    help="Temp directory for user")
        
    def translate(self, text):
        try:
            result = self.translator.translate(text)
        except:
            result = "Null"
        return result
    
    def utc_to_local(self, utc_timestamp, local_tz):
        try:
            local_dt = utc_timestamp.replace(tzinfo=pytz.utc).astimezone(local_tz)
            #sys.stderr.write("LOCAL_DT:"+repr(local_dt)+"\n")
            result = local_tz.normalize(local_dt)
            #sys.stderr.write("RESULT ====> "+repr(result)+"\n")
        except:
            result = None
        return result
    
    def get_time_of_day(self, hour):
        for key in self.time_of_day.keys():
            if hour in self.time_of_day[key]:
                return key

    def mapper_translate(self, _, line_):
        
        tzFile = str(self.options.tzFile) 
        
        tz_dict = dict()
        
        #sys.stderr.write("About to OPEN tzFILE"+"\n")
        
        with open(tzFile,'r') as f:
            
            #sys.stderr.write("OPENED tzFILE"+"\n")
            
            lines     = f.readlines()
            
            tz_dict   = ast.literal_eval(lines[0])
            
        #sys.stderr.write("LINE_ ====> "+repr(line_.split("\t"))+"\n")
    
        if len(line_.split('\t')) > self.min_line_length:
            
            line      = line_.split("\t")
            user_id   = list(line)[0]
            value     = [v.replace('"','').strip() for v in line[1:]]
            utc_time  = value[0][:-1]
            traid     = value[1]
            try:
                #artname   = self.translator.translate(value[2]).text
                artname   = value[2]
            except:
                artname   = value[2]
            something = value[3]
            try:
                #tracname  = self.translator.translate(value[4]).text
                tracname  = value[4]
            except:
                tracname  = value[4]
            session_window = value[5]
            gender    = value[6]
            age       = value[7]
            country   = value[8]
            registered= value[9] 
            reg_date  = value[10]
                
                
            try:
                local_tz  = pytz.timezone(tz_dict[country])
            except:
                local_tz  = None
            try:
                utc    = datetime.strptime(utc_time, '%Y-%m-%dT%H:%M:%S')
            except: 
                utc    = None
                
            try:
                event_time = self.utc_to_local(utc, local_tz)
            except: 
                event_time = None
                    
            try:
                time_diff  = event_time.utcoffset()
                local_time = event_time + time_diff
            except:
                local_time = "Null"
                
            try:
                day_of_week = self.day_of_week[local_time.weekday()]
            except:
                day_of_week = "Null"
                    
            try:
                time_of_day = self.get_time_of_day(local_time.hour)
            except:
                time_of_day = "Null"
                    
            #sys.stderr.write(" utc_time: " + repr(str(utc_time)) + " traid:  "    + repr(str(traid))     + " artname: "     + repr(str(artname)) +"\n")
            #sys.stderr.write(" artname:  " + repr(str(artname))  + " something: " + repr(str(something)) +  "\n")
            #sys.stderr.write(" tracname: " + repr(str(tracname)) + " session_window: " + repr(str(session_window)) + " gender: " + repr(str(gender)) + "\n")
            #sys.stderr.write(" age: "      + repr(str(age))      + " country:"   + repr(str(country))    + " registered: "  + repr(str(registered))  + "\n")
            #sys.stderr.write(" reg_date: " + repr(str(reg_date)) + " local_tz: " + repr(str(local_tz))   + " local_time: "  + str(local_time) + " time_of_day: " + str(time_of_day) + " day_of_week: " +  str(day_of_week)+"\n")
            #sys.stderr.write("============================================================================================================================")
            #sys.stderr.write("============================================================================================================================")
                    
            value_ = []
                
            for x in [utc_time, traid, artname, something, tracname, session_window, gender, age, country, registered, reg_date, local_tz, local_time, time_of_day, day_of_week]:
                try:
                    value_.append(repr(x))
                except:
                    value_.append("Null")
                        
            #sys.stderr.write("\t".join([repr(x) for x in value_]))
                
            try:
                yield user_id, value_

            except:
                yield user_id, None
        
        




    def steps(self):
        
        JOBCONF = {
            'mapreduce.job.maps': self.options.num_mappers,
            'mapreduce.job.reduces': self.options.num_reducers,
            'mapreduce.partition.keypartitioner.options': '-k1',
            'mapreduce.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        JOBCONF1 = {
            'mapreduce.job.maps': self.options.num_mappers,
            'mapred.reduce.tasks': self.options.num_reducers,
            'mapreduce.partition.keypartitioner.options': '-k1',
            'mapreduce.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        
        return [MRStep(mapper=self.mapper_translate,
                       jobconf=JOBCONF)]
       

if __name__ == '__main__':
    MRTranslate.run()


Overwriting MRTranslate.py


In [35]:
!head -n 5 data/Session_data_1k.csv

userid	timestamp	artid	artname_	traid	traname_	session_window	gender	age	country	registered	reg_date
user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007	23	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13


In [36]:
!ls -alrth  data/Session_data_*

-rw-r--r-- 1 hduser hadoop 153K Sep 16 11:15 data/Session_data_1k.csv
-rw-r--r-- 1 hduser hadoop 1.6M Sep 16 11:15 data/Session_data_10k.csv
-rw-r--r-- 1 hduser hadoop  17M Sep 16 11:15 data/Session_data_100k.csv
-rw-r--r-- 1 hduser hadoop 1.4K Sep 16 12:45 data/Session_data_10.csv
-rw-r--r-- 1 hduser hadoop 164M Sep 16 20:17 data/Session_data_1M.csv


In [37]:
!export PATH=$PATH:/usr/local/hadoop/bin:/usr/local/hadoop/sbin

In [38]:
!export HADOOP_opts="-Dhadoop.tmp.dir=/path/"

In [39]:
TRANSLATE_PY ='/data_data/session_length/MRTranslate.py'

In [40]:
JAR_FILE = '/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.0.1.jar'

In [41]:
USER = !whoami
USER = 'w210'
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [42]:
OUTPUT_DIR = "{OUTPUT_PATH_BASE}/data/output" 

In [43]:
TEST_1k = "{OUTPUT_PATH_BASE}/data/Session_data_1k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_10k = "{OUTPUT_PATH_BASE}/data/Session_data_10k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_100k = "{OUTPUT_PATH_BASE}/data/Session_data_100k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_1M = "{OUTPUT_PATH_BASE}/data/Session_data_1M.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
FULL_DATA = "{OUTPUT_PATH_BASE}/data/Session_data.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)

In [44]:
!hdfs dfs -chgrp hadoop /user/w210/data/Session*

In [45]:
!hdfs dfs -ls /user/w210/data/Session*

-rw-r--r--   1 hduser hadoop 3298819885 2018-09-18 16:53 /user/w210/data/Session_data.csv
-rw-r--r--   1 hduser hadoop   16905208 2018-09-18 16:53 /user/w210/data/Session_data_100k.csv
-rw-r--r--   1 hduser hadoop    1610078 2018-09-18 16:53 /user/w210/data/Session_data_10k.csv
-rw-r--r--   1 hduser hadoop     156293 2018-09-18 16:53 /user/w210/data/Session_data_1k.csv


In [46]:
!echo {TEST_1k}

/user/w210/data/Session_data_1k.csv


In [47]:
!hdfs dfs -ls {TEST_1k}

-rw-r--r--   1 hduser hadoop     156293 2018-09-18 16:53 /user/w210/data/Session_data_1k.csv


In [48]:
!ls -alrth country_capital*

-rw-r--r-- 1 hduser hadoop 6.3K Sep 16 16:07 country_capital_city_tz
-rw-r--r-- 1 hduser hadoop 6.1K Oct 20 13:13 country_capital_city_tz.json


In [49]:
TZ_FILE='/user/w210/data/country_capital_city_tz.json'

In [50]:
INPUT_FILE='/user/w210/data/Session_data_1M.csv'

In [51]:
TMP_DIR='/user/hduser/tmp'

In [52]:
!ls -alrth data

total 3.3G
-rw-r--r--  1 hduser hadoop 3.1G Sep 15 01:03 Session_data.csv
drwxr-xr-x  2 hduser hadoop 4.0K Sep 15 01:03 tmp
-rw-r--r--  1 hduser hadoop   30 Sep 15 04:37 timezones.csv
-rw-r--r--  1 hduser hadoop    0 Sep 15 15:20 test.csv
-rw-r--r--  1 hduser hadoop 153K Sep 16 11:15 Session_data_1k.csv
-rw-r--r--  1 hduser hadoop 1.6M Sep 16 11:15 Session_data_10k.csv
-rw-r--r--  1 hduser hadoop  17M Sep 16 11:15 Session_data_100k.csv
-rw-r--r--  1 hduser hadoop 1.4K Sep 16 12:45 Session_data_10.csv
drwxr-xr-x  4 hduser hadoop 4.0K Sep 16 20:17 .
-rw-r--r--  1 hduser hadoop 164M Sep 16 20:17 Session_data_1M.csv
drwxr-xr-x  4 hduser hadoop 4.0K Sep 28 19:19 lastFM
drwxr-xr-x 14 hduser hadoop 4.0K Oct 20 13:11 ..


In [53]:
#!hdfs dfs -copyFromLocal data/Session_data.csv /user/hduser/

In [54]:
#!hdfs dfs -rmdir /user/hdfs/output
#!hdfs dfs -ls /user/hduser

In [55]:
#!hdfs dfs -rm -r /user/hduser/output
#!hdfs dfs -ls /user/hdfs

In [56]:
#!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.0.1.jar \
#    -D mapreduce.job.maps=10 \
#    -D hadoop.tmp.dir='/user/hduser/tmp' \
#    -files MRTranslate.py,country_capital_city_tz.json \
#    -input '/user/hduser/Session_data_1M.csv' \
#    -output '/user/hduser/output' \
#    -numReduceTasks 10 \
#    -cmdenv tzFile='country_capital_city_tz.json' \
#    -cmdenv PATH={PATH}

In [57]:
#!hadoop fs -rm -r /user/hdfs/output
#!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.0.1.jar \
#    -D mapreduce.job.maps=10 \
#    -D hadoop.tmp.dir='/user/hduser/tmp' \
#    -files MRTranslate.py,country_capital_city_tz.json\
#    -input '/user/hduser/Session_data_1M.csv' \
#    -output '/user/hduser/output' \
#    -numReduceTasks 10 \
#    -cmdenv tzFile='country_capital_city_tz.json' \
#    -cmdenv PATH={PATH}

In [58]:
!rm /data_data/session_length/session_data_10_is_holiday.tsv
#!rm -R /tmp/MRtranslate*
!python MRTranslate.py ./data/Session_data_10.csv  --tzFile=/data_data/session_length/country_capital_city_tz.json --tmp_dir=/data_data/session_length/hadoop/tmp  > session_data_10_is_holiday.tsv

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
Using configs in /home/hduser/.mrjob.conf
Running step 1 of 1...
Creating temp directory /data_data/session_length/hadoop/tmp/MRTranslate.hduser.20181020.131340.518880
job output is in /data_data/session_length/hadoop/tmp/MRTranslate.hduser.20181020.131340.518880/output
Streaming final output from /data_data/session_length/hadoop/tmp/MRTranslate.hduser.20181020.131340.518880/output...
Removing temp directory /data_data/session_length/hadoop/tmp/MRTranslate.hduser.20181020.131340.518880...


In [59]:
!ls -alrth session_data_10_is_holiday.tsv

-rw-r--r-- 1 hduser hadoop 0 Oct 20 13:13 session_data_10_is_holiday.tsv


In [60]:
!head session_data_10_is_holiday.tsv

In [62]:
#!rm /data_data/session_length/session_data_1M_is_holiday.tsv
#!rm -R /tmp/MRtranslate*
#!python MRTranslate.py ./data/Session_data_1M.csv  --tzFile=/data_data/session_length/country_capital_city_tz.json --tmp_dir=/data_data/session_length/hadoop/tmp  > session_data_1M_is_holiday.tsv

In [1]:
!head session_data_1M.tsv

head: cannot open 'session_data_1M.tsv' for reading: No such file or directory


In [64]:
!pwd

/data_data/session_length


## IMPORT HOLIDAY DATA FOR EACH COUNTRY

In [65]:
import pandas as pd
from lxml import html
from lxml.html import parse
from urllib2 import urlopen

url = "https://www.timeanddate.com/holidays/japan/2008"

parsed = parse(urlopen(url))

doc = parsed.getroot()

In [66]:
holidays_dict = {
"	Angola	":"	AO	",
"	Austria	":"	AT	",
"	Australia	":"	AU	",
"	Aruba	":"	AW	",
"	Falkland Islands	":"	AX	",
"	Bosnia and Herzegovina	":"	BA	",
"	Belgium	":"	BE	",
"	Bulgaria	":"	BG	",
"	Bolivia	":"	BO	",
"	Brazil	":"	BR	",
"	The Bahamas	":"	BS	",
"	Canada	":"	CA	",
"	Switzerland	":"	CH	",
"	China	":"	CN	",
"	Colombia	":"	CO	",
"	Costa Rica	":"	CR	",
"	Cuba	":"	CU	",
"	Czech Republic	":"	CZ	",
"	Germany	":"	DE	",
"	Denmark	":"	DK	",
"	Dominican Republic	":"	DO	",
"	Ecuador	":"	EC	",
"	Spain	":"	ES	",
"	Finland	":"	FI	",
"	France	":"	FR	",
"	Alsace	":"	FR	",
"	United Kingdom	":"	GB	",
"	England	":"	GB	",
"	Northern Ireland	":"	GB	",
"	Scotland	":"	GB	",
"	Wales	":"	GB	",
"	Greece	":"	GR	",
"	Guatemala	":"	GT	",
"	Hong Kong	":"	HK	",
"	Honduras	":"	HN	",
"	Croatia	":"	HR	",
"	Hungary	":"	HU	",
"	Indonesia	":"	ID	",
"	Ireland	":"	IE	",
"	India	":"	IN	",
"	Israel	":"	IL	",
"	Iceland	":"	IS	",
"	Italy	":"	IT	",
"	Japan	":"	JP	",
"	Kazakhstan	":"	KZ	",
"	Lesotho	":"	LS	",
"	Luxembourg	":"	LU	",
"	Madagascar	":"	MG	",
"	Martinique	":"	MQ	",
"	Malta	":"	MT	",
"	Mauritius	":"	MU	",
"	Mexico	":"	MX	",
"	Mozambique	":"	MZ	",
"	Nigeria	":"	NG	",
"	Netherlands	":"	NL	",
"	Norway	":"	NO	",
"	Peru	":"	PE	",
"	Pakistan	":"	PK	",
"	Philippines	":"	PH	",
"	Poland	":"	PL	",
"	Puerto Rico	":"	PR	",
"	Portugal	":"	PT	",
"	Paraguay	":"	PY	",
"	Reunion	":"	RE	",
"	Romania	":"	RO	",
"	Russia	":"	RU	",
"	Seychelles	":"	SC	",
"	Sweden	":"	SE	",
"	Singapore	":"	SG	",
"	Slovenia	":"	SI	",
"	Sao Tome and Principe	":"	ST	",
"	Slovakia	":"	SK	",
"	Tunisia	":"	TN	",
"	Turkey	":"	TR	",
"	Ukraine	":"	UA	",
"	United States	":"	US	",
"	Uruguay	":"	UY	",
"	Venezuela	":"	VE	",
"	South Africa	":"	ZA	",
"	Zambia	":"	ZA	",
"	Venezuela	":"	VE	",
"	South Africa	":"	ZA	",

}

In [67]:
h_dict = dict()
holiday_dict = [h_dict.update({value.strip():key.strip()}) for value, key in holidays_dict.items()]
h_dict

{'Alsace': 'FR',
 'Angola': 'AO',
 'Aruba': 'AW',
 'Australia': 'AU',
 'Austria': 'AT',
 'Belgium': 'BE',
 'Bolivia': 'BO',
 'Bosnia and Herzegovina': 'BA',
 'Brazil': 'BR',
 'Bulgaria': 'BG',
 'Canada': 'CA',
 'China': 'CN',
 'Colombia': 'CO',
 'Costa Rica': 'CR',
 'Croatia': 'HR',
 'Cuba': 'CU',
 'Czech Republic': 'CZ',
 'Denmark': 'DK',
 'Dominican Republic': 'DO',
 'Ecuador': 'EC',
 'England': 'GB',
 'Falkland Islands': 'AX',
 'Finland': 'FI',
 'France': 'FR',
 'Germany': 'DE',
 'Greece': 'GR',
 'Guatemala': 'GT',
 'Honduras': 'HN',
 'Hong Kong': 'HK',
 'Hungary': 'HU',
 'Iceland': 'IS',
 'India': 'IN',
 'Indonesia': 'ID',
 'Ireland': 'IE',
 'Israel': 'IL',
 'Italy': 'IT',
 'Japan': 'JP',
 'Kazakhstan': 'KZ',
 'Lesotho': 'LS',
 'Luxembourg': 'LU',
 'Madagascar': 'MG',
 'Malta': 'MT',
 'Martinique': 'MQ',
 'Mauritius': 'MU',
 'Mexico': 'MX',
 'Mozambique': 'MZ',
 'Netherlands': 'NL',
 'Nigeria': 'NG',
 'Northern Ireland': 'GB',
 'Norway': 'NO',
 'Pakistan': 'PK',
 'Paraguay': 'PY',


In [68]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x7f00f2cf1838>,
 <Element a at 0x7f00f2cf1890>,
 <Element a at 0x7f00f2cf18e8>,
 <Element a at 0x7f00f2cf1940>,
 <Element a at 0x7f00f2cf1998>]

In [69]:
d = pd.read_html(url)
d[0][:10]

,Date,Weekday,Holiday Name,Holiday Type
0,Jan 1,Tuesday,New Year's Day,National holiday
1,Jan 2,Wednesday,January 2 Bank Holiday,Bank holiday
2,Jan 3,Thursday,January 3 Bank Holiday,Bank holiday
3,Jan 14,Monday,Coming of Age Day,National holiday
4,Feb 11,Monday,National Foundation Day,National holiday
5,Feb 14,Thursday,Valentine's Day,Observance
6,Mar 3,Monday,Dolls' Festival/Girls' Festival,Observance
7,Mar 20,Thursday,March Equinox,Season
8,Mar 20,Thursday,Spring Equinox,National holiday
9,Apr 29,Tuesday,Shōwa Day,National holiday


In [70]:
!whoami

hduser


In [71]:
holiday_df = pd.DataFrame([],columns=['Date','Weekday','Holiday Name','Holiday Type','Year','Country'])
holiday_df

,Date,Weekday,Holiday Name,Holiday Type,Year,Country


In [72]:
_countries_ = {"	Afghanistan	":	"Null"	,
"	Albania	":	"Null"	,
"	Algeria	":	"Null"	,
"	American Samoa	":	"Null"	,
"	Andorra	":	"Null"	,
"	Angola	":	"Null"	,
"	Anguilla	":	"Null"	,
"	Antigua and Barbuda	":	"Null"	,
"	Argentina	":	"Null"	,
"	Armenia	":	"Null"	,
"	Aruba	":	"Null"	,
"	Australia	":	"Null"	,
"	Austria	":	"Null"	,
"	Azerbaijan	":	"Null"	,
"	Bahrain	":	"Null"	,
"	Bangladesh	":	"Null"	,
"	Barbados	":	"Null"	,
"	Belarus	":	"Null"	,
"	Belgium	":	"Null"	,
"	Belize	":	"Null"	,
"	Benin	":	"Null"	,
"	Bermuda	":	"Null"	,
"	Bhutan	":	"Null"	,
"	Bolivia	":	"Null"	,
"	Bosnia and Herzegovina	":	"Null"	,
"	Botswana	":	"Null"	,
"	Brazil	":	"Null"	,
"	British Virgin Islands	":	"Null"	,
"	British Virgin Islands	":	"Null"	,
"	Brunei	":	"Null"	,
"	Bulgaria	":	"Null"	,
"	Burkina Faso	":	"Null"	,
"	Burundi	":	"Null"	,
"	Cabo Verde	":	"Null"	,
"	Cambodia	":	"Null"	,
"	Cameroon	":	"Null"	,
"	Canada	":	"Null"	,
"	Cayman Islands	":	"Null"	,
"	Central African Republic	":	"Null"	,
"	Chad	":	"Null"	,
"	Chile	":	"Null"	,
"	China	":	"Null"	,
"	Colombia	":	"Null"	,
"	Comoros	":	"Null"	,
"	Congo	":	"Null"	,
"	Congo Democratic Republic	":	"Null"	,
"	Cook Islands	":	"Null"	,
"	Costa Rica	":	"Null"	,
"	Ivory Coast	":	"Null"	,
"	Croatia	":	"Null"	,
"	Cuba	":	"Null"	,
"	Curacao	":	"Null"	,
"	Cyprus	":	"Null"	,
"	Cyprus	":	"Null"	,
"	Czech Republic	":	"Null"	,
"	Denmark	":	"Null"	,
"	Djibouti	":	"Null"	,
"	Dominica	":	"Null"	,
"	Dominican Republic	":	"Null"	,
"	East Timor	":	"Null"	,
"	Ecuador	":	"Null"	,
"	Egypt	":	"Null"	,
"	El Salvador	":	"Null"	,
"	Equatorial Guinea	":	"Null"	,
"	Eritrea	":	"Null"	,
"	Estonia	":	"Null"	,
"	eSwatini	":	"Null"	,
"	Ethiopia	":	"Null"	,
"	Falkland Islands	":	"Null"	,
"	Faroe Islands	":	"Null"	,
"	Fiji	":	"Null"	,
"	Finland	":	"Null"	,
"	France	":	"Null"	,
"	French Guiana	":	"Null"	,
"	French Polynesia	":	"Null"	,
"	Gabon	":	"Null"	,
"	Gambia	":	"Null"	,
"	Georgia	":	"Null"	,
"	Georgia	":	"Null"	,
"	Germany	":	"Null"	,
"	Ghana	":	"Null"	,
"	Gibraltar	":	"Null"	,
"	Greece	":	"Null"	,
"	Greenland	":	"Null"	,
"	Grenada	":	"Null"	,
"	Guam	":	"Null"	,
"	Guatemala	":	"Null"	,
"	Guernsey	":	"Null"	,
"	Guinea	":	"Null"	,
"	Guinea-Bissau	":	"Null"	,
"	Guyana	":	"Null"	,
"	Haiti	":	"Null"	,
"	Holy See	":	"Null"	,
"	Honduras	":	"Null"	,
"	Hong Kong	":	"Null"	,
"	Hungary	":	"Null"	,
"	Iceland	":	"Null"	,
"	India	":	"Null"	,
"	Indonesia	":	"Null"	,
"	Iran	":	"Null"	,
"	Iraq	":	"Null"	,
"	Ireland	":	"Null"	,
"	Isle of Man	":	"Null"	,
"	Israel	":	"Null"	,
"	Italy	":	"Null"	,
"	Jamaica	":	"Null"	,
"	Japan	":	"Null"	,
"	Jersey	":	"Null"	,
"	Jordan	":	"Null"	,
"	Kazakhstan	":	"Null"	,
"	Kenya	":	"Null"	,
"	Kiribati	":	"Null"	,
"	Kosovo	":	"Null"	,
"	Kuwait	":	"Null"	,
"	Kyrgyzstan	":	"Null"	,
"	Laos	":	"Null"	,
"	Latvia	":	"Null"	,
"	Lebanon	":	"Null"	,
"	Lesotho	":	"Null"	,
"	Liberia	":	"Null"	,
"	Libya	":	"Null"	,
"	Liechtenstein	":	"Null"	,
"	Lithuania	":	"Null"	,
"	Luxembourg	":	"Null"	,
"	Macau	":	"Null"	,
"	Macedonia	":	"Null"	,
"	Madagascar	":	"Null"	,
"	Malawi	":	"Null"	,
"	Malaysia	":	"Null"	,
"	Maldives	":	"Null"	,
"	Mali	":	"Null"	,
"	Malta	":	"Null"	,
"	Marshall Islands	":	"Null"	,
"	Martinique	":	"Null"	,
"	Mauritania	":	"Null"	,
"	Mauritius	":	"Null"	,
"	Mayotte	":	"Null"	,
"	Mexico	":	"Null"	,
"	Micronesia	":	"Null"	,
"	Moldova	":	"Null"	,
"	Monaco	":	"Null"	,
"	Mongolia	":	"Null"	,
"	Montenegro	":	"Null"	,
"	Montserrat	":	"Null"	,
"	Morocco	":	"Null"	,
"	Mozambique	":	"Null"	,
"	Myanmar	":	"Null"	,
"	Namibia	":	"Null"	,
"	Nauru	":	"Null"	,
"	Nepal	":	"Null"	,
"	Netherlands	":	"Null"	,
"	New Caledonia	":	"Null"	,
"	New Zealand	":	"Null"	,
"	Nicaragua	":	"Null"	,
"	Niger	":	"Null"	,
"	Nigeria	":	"Null"	,
"	North Korea	":	"Null"	,
"	Northern Mariana Islands	":	"Null"	,
"	Norway	":	"Null"	,
"	Oman	":	"Null"	,
"	Pakistan	":	"Null"	,
"	Palau	":	"Null"	,
"	Panama	":	"Null"	,
"	Papua New Guinea	":	"Null"	,
"	Paraguay	":	"Null"	,
"	Peru	":	"Null"	,
"	Philippines	":	"Null"	,
"	Poland	":	"Null"	,
"	Portugal	":	"Null"	,
"	Puerto Rico	":	"Null"	,
"	Qatar	":	"Null"	,
"	Reunion	":	"Null"	,
"	Romania	":	"Null"	,
"	Russia	":	"Null"	,
"	Russia	":	"Null"	,
"	Rwanda	":	"Null"	,
"	Saint Helena	":	"Null"	,
"	Saint Kitts and Nevis	":	"Null"	,
"	Saint Lucia	":	"Null"	,
"	Saint Martin	":	"Null"	,
"	Saint Pierre and Miquelon	":	"Null"	,
"	Saint Vincent and the Grenadines	":	"Null"	,
"	Samoa	":	"Null"	,
"	San Marino	":	"Null"	,
"	Sao Tome and Principe	":	"Null"	,
"	Saudi Arabia	":	"Null"	,
"	Senegal	":	"Null"	,
"	Serbia	":	"Null"	,
"	Seychelles	":	"Null"	,
"	Sierra Leone	":	"Null"	,
"	Singapore	":	"Null"	,
"	Sint Maarten	":	"Null"	,
"	Slovakia	":	"Null"	,
"	Slovenia	":	"Null"	,
"	Solomon Islands	":	"Null"	,
"	Somalia	":	"Null"	,
"	South Africa	":	"Null"	,
"	South Korea	":	"Null"	,
"	South Sudan	":	"Null"	,
"	Spain	":	"Null"	,
"	Sri Lanka	":	"Null"	,
"	St. Barts	":	"Null"	,
"	Sudan	":	"Null"	,
"	Suriname	":	"Null"	,
"	Sweden	":	"Null"	,
"	Switzerland	":	"Null"	,
"	Syria	":	"Null"	,
"	Taiwan	":	"Null"	,
"	Tajikistan	":	"Null"	,
"	Tanzania	":	"Null"	,
"	Thailand	":	"Null"	,
"	The Bahamas	":	"Null"	,
"	Togo	":	"Null"	,
"	Tonga	":	"Null"	,
"	Trinidad and Tobago	":	"Null"	,
"	Tunisia	":	"Null"	,
"	Turkey	":	"Null"	,
"	Turkey	":	"Null"	,
"	Turkmenistan	":	"Null"	,
"	Turks and Caicos Islands	":	"Null"	,
"	Tuvalu	":	"Null"	,
"	Uganda	":	"Null"	,
"	Ukraine	":	"Null"	,
"	United Arab Emirates	":	"Null"	,
"	United Kingdom	":	"Null"	,
"	United States	":	"Null"	,
"	Uruguay	":	"Null"	,
"	US Virgin Islands	":	"Null"	,
"	Uzbekistan	":	"Null"	,
"	Vanuatu	":	"Null"	,
"	Venezuela	":	"Null"	,
"	Vietnam	":	"Null"	,
"	Wallis and Futuna	":	"Null"	,
"	Yemen	":	"Null"	,
"	Zambia	":	"Null"	,
"	Zimbabwe	":	"Null"	}

#### Generate list of countries and create list of alternate country_names [exceptions]

In [73]:
_countries = [c.lower().strip() for c in _countries_.keys()]
_countries[:5]

['palau', 'saint pierre and miquelon', 'cameroon', 'kosovo', 'zimbabwe']

#### List of alternative country names

In [74]:
exceptions = {"czech republic":"czech","bosnia and herzegovina":"bosnia","alsace":"france","eswatini":"swaziland",
              "united states":"us","united kingdom":"uk","wales":"uk","scotland":"uk", "northern ireland":"uk",
              "congo democratic republic":"dr-congo","equatorial guinea":"guineaecuatorial", "congo":"republic-of-the-congo",
              "east timor":"timor-leste", "trinidad and tobago":"trinidad","the bahamas":"bahamas", "comoros":"comores",
              "us virgin islands":"united-states-virgin-islands","cabo verde":"cape-verde","st. barts":"saint-barthelemy",
              "russian federation":"russia", "cote d'ivoire":"ivory coast", "korea, democratic people's republic of":"north korea",
              "congo, the democratic republic of the":"dr-congo"}


In [75]:
years = [2004, 2005, 2006, 2007, 2008, 2009]

In [76]:
base_url   = "https://www.timeanddate.com/holidays/"

countries  = [ c.lower() for c in h_dict.keys()]


for country in _countries:
    
    for year in years:
        if country.lower().startswith("the"):
            country == country.lower().replace("the ","")
            print("COUNTRY: {} replaced THE ".format(country))
        else:
            if country in exceptions:
                country = exceptions[country]
                print("COUNTRY: {}  found in EXCEPTIONS DICT".format(country))
            elif " " in country.lower():
                country = "-".join(country.lower().split(" "))
                print("COUNTRY: {} not found CONCATENATED NAME".format(country))
            else:
                pass
        try:
            url = base_url + str(country).lower() + "/" + str(year)
            d = pd.read_html(url)
            d[0]["Year"] = int(year)
            d[0]["Country"] = country
            print("Found holidays for:\t{}".format(country))
            holiday_df = pd.concat([holiday_df, d[0]])
        except:
            print("COUNTRY: {} not found @timeanddate.com".format(country))
            print("COUNTRY: {} not found @timeanddate.com II".format(country))

Found holidays for:	palau
Found holidays for:	palau
Found holidays for:	palau
Found holidays for:	palau
Found holidays for:	palau
Found holidays for:	palau
COUNTRY: saint-pierre-and-miquelon not found CONCATENATED NAME
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	saint-pierre-and-miquelon
Found holidays for:	cameroon
Found holidays for:	cameroon
Found holidays for:	cameroon
Found holidays for:	cameroon
Found holidays for:	cameroon
Found holidays for:	cameroon
Found holidays for:	kosovo
Found holidays for:	kosovo
Found holidays for:	kosovo
Found holidays for:	kosovo
Found holidays for:	kosovo
Found holidays for:	kosovo
Found holidays for:	zimbabwe
Found holidays for:	zimbabwe
Found holidays for:	zimbabwe
Found holidays for:	zimbabwe
Found holidays for:	zimbabwe
Found holidays for:	zimb

Found holidays for:	montenegro
Found holidays for:	montenegro
Found holidays for:	montenegro
Found holidays for:	montenegro
Found holidays for:	montenegro
Found holidays for:	montenegro
COUNTRY: sierra-leone not found CONCATENATED NAME
Found holidays for:	sierra-leone
Found holidays for:	sierra-leone
Found holidays for:	sierra-leone
Found holidays for:	sierra-leone
Found holidays for:	sierra-leone
Found holidays for:	sierra-leone
COUNTRY: swaziland  found in EXCEPTIONS DICT
Found holidays for:	swaziland
Found holidays for:	swaziland
Found holidays for:	swaziland
Found holidays for:	swaziland
Found holidays for:	swaziland
Found holidays for:	swaziland
COUNTRY: saint-vincent-and-the-grenadines not found CONCATENATED NAME
Found holidays for:	saint-vincent-and-the-grenadines
Found holidays for:	saint-vincent-and-the-grenadines
Found holidays for:	saint-vincent-and-the-grenadines
Found holidays for:	saint-vincent-and-the-grenadines
Found holidays for:	saint-vincent-and-the-grenadines
Found 

Found holidays for:	new-zealand
Found holidays for:	new-zealand
Found holidays for:	new-zealand
Found holidays for:	new-zealand
Found holidays for:	new-zealand
Found holidays for:	new-zealand
Found holidays for:	ghana
Found holidays for:	ghana
Found holidays for:	ghana
Found holidays for:	ghana
Found holidays for:	ghana
Found holidays for:	ghana
Found holidays for:	france
Found holidays for:	france
Found holidays for:	france
Found holidays for:	france
Found holidays for:	france
Found holidays for:	france
Found holidays for:	india
Found holidays for:	india
Found holidays for:	india
Found holidays for:	india
Found holidays for:	india
Found holidays for:	india
Found holidays for:	serbia
Found holidays for:	serbia
Found holidays for:	serbia
Found holidays for:	serbia
Found holidays for:	serbia
Found holidays for:	serbia
Found holidays for:	russia
Found holidays for:	russia
Found holidays for:	russia
Found holidays for:	russia
Found holidays for:	russia
Found holidays for:	russia
Found holi

Found holidays for:	taiwan
Found holidays for:	mayotte
Found holidays for:	mayotte
Found holidays for:	mayotte
Found holidays for:	mayotte
Found holidays for:	mayotte
Found holidays for:	mayotte
Found holidays for:	guyana
Found holidays for:	guyana
Found holidays for:	guyana
Found holidays for:	guyana
Found holidays for:	guyana
Found holidays for:	guyana
Found holidays for:	curacao
Found holidays for:	curacao
Found holidays for:	curacao
Found holidays for:	curacao
Found holidays for:	curacao
Found holidays for:	curacao
Found holidays for:	kuwait
Found holidays for:	kuwait
Found holidays for:	kuwait
Found holidays for:	kuwait
Found holidays for:	kuwait
Found holidays for:	kuwait
Found holidays for:	indonesia
Found holidays for:	indonesia
Found holidays for:	indonesia
Found holidays for:	indonesia
Found holidays for:	indonesia
Found holidays for:	indonesia
Found holidays for:	iraq
Found holidays for:	iraq
Found holidays for:	iraq
Found holidays for:	iraq
Found holidays for:	iraq
Found ho

Found holidays for:	afghanistan
Found holidays for:	afghanistan
Found holidays for:	afghanistan
Found holidays for:	afghanistan
COUNTRY: faroe-islands not found CONCATENATED NAME
Found holidays for:	faroe-islands
Found holidays for:	faroe-islands
Found holidays for:	faroe-islands
Found holidays for:	faroe-islands
Found holidays for:	faroe-islands
Found holidays for:	faroe-islands
COUNTRY: comores  found in EXCEPTIONS DICT
Found holidays for:	comores
Found holidays for:	comores
Found holidays for:	comores
Found holidays for:	comores
Found holidays for:	comores
Found holidays for:	comores
Found holidays for:	turkmenistan
Found holidays for:	turkmenistan
Found holidays for:	turkmenistan
Found holidays for:	turkmenistan
Found holidays for:	turkmenistan
Found holidays for:	turkmenistan
COUNTRY: united-states-virgin-islands  found in EXCEPTIONS DICT
Found holidays for:	united-states-virgin-islands
Found holidays for:	united-states-virgin-islands
Found holidays for:	united-states-virgin-islan

Found holidays for:	honduras
Found holidays for:	honduras
Found holidays for:	honduras
Found holidays for:	honduras
Found holidays for:	honduras
Found holidays for:	honduras
Found holidays for:	ecuador
Found holidays for:	ecuador
Found holidays for:	ecuador
Found holidays for:	ecuador
Found holidays for:	ecuador
Found holidays for:	ecuador
COUNTRY: saint-lucia not found CONCATENATED NAME
Found holidays for:	saint-lucia
Found holidays for:	saint-lucia
Found holidays for:	saint-lucia
Found holidays for:	saint-lucia
Found holidays for:	saint-lucia
Found holidays for:	saint-lucia
COUNTRY: sao-tome-and-principe not found CONCATENATED NAME
Found holidays for:	sao-tome-and-principe
Found holidays for:	sao-tome-and-principe
Found holidays for:	sao-tome-and-principe
Found holidays for:	sao-tome-and-principe
Found holidays for:	sao-tome-and-principe
Found holidays for:	sao-tome-and-principe
Found holidays for:	mexico
Found holidays for:	mexico
Found holidays for:	mexico
Found holidays for:	mexic

#### QA holidays generated by script

In [77]:
holiday_df.head(100)

,Country,Date,Flag Day,Holiday Name,Holiday Type,Weekday,Where It is Observed,Year
0,palau,Jan 1,NaN,New Year's Day,Public Holiday,Thursday,NaN,2004
1,palau,Mar 15,NaN,Youth Day,Public Holiday,Monday,NaN,2004
2,palau,Mar 20,NaN,March Equinox,Season,Saturday,NaN,2004
3,palau,May 5,NaN,Senior Citizens Day,Public Holiday,Wednesday,NaN,2004
4,palau,Jun 1,NaN,President's Day,Public Holiday,Tuesday,NaN,2004
5,palau,Jun 21,NaN,June Solstice,Season,Monday,NaN,2004
6,palau,Jul 9,NaN,Constitution Day,Public Holiday,Friday,NaN,2004
7,palau,Sep 6,NaN,Labor Day,Public Holiday,Monday,NaN,2004
8,palau,Sep 22,NaN,September Equinox,Season,Wednesday,NaN,2004
9,palau,Oct 1,NaN,Independence Day,Public Holiday,Friday,NaN,2004


In [78]:
holiday_df.columns = [x.lower() for x in holiday_df.columns]

In [79]:
years

[2004, 2005, 2006, 2007, 2008, 2009]

In [80]:
holiday_df.iloc[15000:,:]

,country,date,flag day,holiday name,holiday type,weekday,where it is observed,year
13,norway,Mar 24,NaN,Easter Monday,National holiday,Monday,NaN,2008
14,norway,Mar 30,NaN,Daylight Saving Time starts,Clock change/Daylight Saving Time,Sunday,NaN,2008
15,norway,May 1,NaN,Ascension Day,National holiday,Thursday,NaN,2008
16,norway,May 1,NaN,Labor day,"National holiday, Flag day",Thursday,NaN,2008
17,norway,May 8,NaN,Day of liberation,"Observance, Flag day",Thursday,NaN,2008
18,norway,May 10,NaN,Pentecost Eve,Observance,Saturday,NaN,2008
19,norway,May 11,NaN,Whit Sunday,"National holiday, Flag day",Sunday,NaN,2008
20,norway,May 12,NaN,Whit Monday,National holiday,Monday,NaN,2008
21,norway,May 17,NaN,17 May Constitution Day (1814),"National holiday, Flag day",Saturday,NaN,2008
22,norway,Jun 7,NaN,Dissolution of union with Sweden (1905),Flag day,Saturday,NaN,2008


In [81]:
holiday_DF = holiday_df.reset_index().set_index(['country','year','date']).iloc[:,1:4].reset_index().iloc[:,[0,1,2,4,5]]
holiday_DF[:5]

,country,year,date,holiday name,holiday type
0,palau,2004,Jan 1,New Year's Day,Public Holiday
1,palau,2004,Mar 15,Youth Day,Public Holiday
2,palau,2004,Mar 20,March Equinox,Season
3,palau,2004,May 5,Senior Citizens Day,Public Holiday
4,palau,2004,Jun 1,President's Day,Public Holiday


In [82]:
holiday_DF['_date'] = holiday_DF.apply(lambda row: "-".join(str(row["date"]).split(" "))+"-"+str(row["year"]), axis=1)
holiday_DF[:5]

,country,year,date,holiday name,holiday type,_date
0,palau,2004,Jan 1,New Year's Day,Public Holiday,Jan-1-2004
1,palau,2004,Mar 15,Youth Day,Public Holiday,Mar-15-2004
2,palau,2004,Mar 20,March Equinox,Season,Mar-20-2004
3,palau,2004,May 5,Senior Citizens Day,Public Holiday,May-5-2004
4,palau,2004,Jun 1,President's Day,Public Holiday,Jun-1-2004


In [83]:
holiday_DF = holiday_DF.iloc[:,[0,5]]
holiday_DF["is_holiday"] = 1
holiday_DF[:5]

,country,_date,is_holiday
0,palau,Jan-1-2004,1
1,palau,Mar-15-2004,1
2,palau,Mar-20-2004,1
3,palau,May-5-2004,1
4,palau,Jun-1-2004,1


In [84]:
#holiday_DF = holiday_DF.drop_duplicates(subset=['Country', '_Date'], keep=False)
#holiday_DF[:5]

In [85]:
holiday_DF.columns = ["_country", "date", "is_holiday"]
holiday_DF = holiday_DF.drop_duplicates(subset=['_country', 'date'])
holiday_DF[:5]

,_country,date,is_holiday
0,palau,Jan-1-2004,1
1,palau,Mar-15-2004,1
2,palau,Mar-20-2004,1
3,palau,May-5-2004,1
4,palau,Jun-1-2004,1


In [86]:
 
Holidays_DF = pd.pivot_table(holiday_DF, values = 'is_holiday', index=['date'], columns = '_country',fill_value=0).unstack().reset_index(['_country','date'])
Holidays_DF.columns=['_country','date','is_holiday']

In [87]:
Holidays_DF['country'] = Holidays_DF['_country'].apply(lambda name: ' '.join(name.lower().split('-')))
Holidays_DF[:5]

,_country,date,is_holiday,country
0,algeria,Apr-1-2004,0,algeria
1,algeria,Apr-1-2005,0,algeria
2,algeria,Apr-1-2006,0,algeria
3,algeria,Apr-1-2007,0,algeria
4,algeria,Apr-1-2008,0,algeria


In [88]:
Holidays_DF.sample(10)

,_country,date,is_holiday,country
40480,botswana,Dec-5-2005,0,botswana
50501,cambodia,Apr-1-2009,0,cambodia
80861,dominica,Jul-17-2008,0,dominica
358064,yemen,Dec-13-2009,0,yemen
55089,canada,Dec-14-2004,0,canada
324124,tunisia,Apr-26-2007,0,tunisia
106437,french-polynesia,Mar-12-2006,0,french polynesia
306641,sweden,May-8-2005,0,sweden
9976,anguilla,May-8-2004,0,anguilla
184851,malaysia,Oct-18-2007,0,malaysia


In [89]:
!rm international_holidays.tsv.bak
!mv international_holidays.tsv international_holidays.tsv.bak

In [90]:
Holidays_DF.to_csv('international_holidays.tsv', header=True, sep='\t', encoding='utf-8')

In [91]:
!head international_holidays.tsv

	_country	date	is_holiday	country
0	algeria	Apr-1-2004	0	algeria
1	algeria	Apr-1-2005	0	algeria
2	algeria	Apr-1-2006	0	algeria
3	algeria	Apr-1-2007	0	algeria
4	algeria	Apr-1-2008	0	algeria
5	algeria	Apr-1-2009	0	algeria
6	algeria	Apr-10-2004	0	algeria
7	algeria	Apr-10-2005	0	algeria
8	algeria	Apr-10-2006	0	algeria


#### List of International Holidays created in "Import National Holidays.ipynb"

##### Format of Holidays_DF

Date: for each country we included every day in the year

Country: Every country in the timeanddate.com database is included [same data used in ios/apple]

is_holiday: Flag set to 0 if Date == holiday, otherwise set to 1


In [13]:
Holidays_DF = pd.read_csv('/data_data/session_length/international_holidays.tsv', sep='\t').iloc[:,2:6].set_index(['country','date']).unstack()

In [14]:
Holidays_DF.head(10)

is_holiday                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   \
date                Apr-1-2004 Apr-1-2005 Apr-1-2006 Apr-1-2007 Apr-1-2008 Apr-1-2009 Apr-10-2004 Apr-10-2005 Apr-10-2006 Apr-10-2007 Apr-10-2009 Apr-11-2004 Apr-11-2005 Apr-11-2006 Apr-11-2007 Apr-11-2008 Apr-11-2009 Apr-12-2004 Apr-12-2005 Apr-12-2006 Apr-12-2007 Apr-12-2008 Apr-12-2009 Apr-13-2004 Apr-13-2005 Apr-13-2006 Apr-13-2007 Apr-13-2008 Apr-13-2009 Apr-14-2004 Apr-14-2005 Apr-14-2006 Apr-14-2007 Apr-14-2008 Apr-14-2009 Apr-15-2004 Apr-15-2005 Apr-15-2006 Apr-15-2007 Apr-15-2008 Apr-15-2009 Apr-16-2004 Apr-16-2005 Apr-16-2006 Apr-16-2007 Apr-16-2008 Apr-16-2009 Apr-17-2004 Apr-17-2005 Apr-17-2006 Apr-17-2007 Apr-17-2008 Apr-17-2009 Apr-18-2004 Apr-18-2005 Apr-18-2006 Apr-18-2008 Apr-18-2009 Apr-19-2004 Apr-19-2005 Apr-19-2006 Apr-19-2007 Apr-19-2008 Apr-19-2009 Apr-2-2004 Apr-2-2005 Apr-2-2006 Apr-2-2007 Apr-2-2008 Apr-2-2009 Apr-20-2004 Apr-20-2006 Apr-20-2007 Apr-20-2008 Apr-20-2009 Apr-21-2004 Apr-21-2005 Apr-21-2006 Apr-21-2007 Apr-21-2008 Apr-21-2009 Apr-22-2004   
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
algeria                      0          0          0          0          0          0           0           0           0           0           0           0           0           1           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0          0          0          0          0          0          0           0           0           0           0           0           0           1           0           0           0           0           0   
american samoa               0          

#### You can query Holidays_DF as follows (if not using JOIN)

In [15]:
country = 'algeria'

In [16]:
date = 'Dec-25-2008'

In [17]:
query_prefix = 'is_holiday'

In [18]:
ctry_holidays_dict = dict(Holidays_DF.loc[country])

In [19]:
ctry_holidays_dict['is_holiday',date]

0

In [20]:
int_holidays_dict = dict(Holidays_DF)

In [21]:
int_holidays_dict['is_holiday', date][country]

0

#### NOTE that some countries have alternate names

In [22]:
exceptions = {"czech republic":"czech","bosnia and herzegovina":"bosnia","alsace":"france","eswatini":"swaziland",
              "united states":"us","united kingdom":"uk","wales":"uk","scotland":"uk", "northern ireland":"uk",
              "congo democratic republic":"dr-congo","equatorial guinea":"guineaecuatorial", "congo":"republic-of-the-congo",
              "east timor":"timor-leste", "trinidad and tobago":"trinidad","the bahamas":"bahamas", "comoros":"comores",
              "us virgin islands":"united-states-virgin-islands","cabo verde":"cape-verde","st. barts":"saint-barthelemy",
              "russian federation":"russia", "cote d'ivoire":"ivory coast", "korea, democratic people's republic of":"north korea",
              "congo, the democratic republic of the":"dr-congo"}

In [23]:
countries_with_holidays_dict = int_holidays_dict['is_holiday', date].keys()

In [24]:
def get_holiday_status(country, date):
    
    try:
        date = datetime.strptime(date, '%Y-%m-%d').strftime('%b-%d-%Y')
    except:
        try:
            date = date.strftime('%b-%d-%Y')
        except:
            pass
    try:
    
        country = country.lower()

        if country in countries_with_holidays_dict:

            return int_holidays_dict['is_holiday', date][country]

        else:

            if country.lower().startswith("the"):

                country = country.lower().replace("the ","")

                try:
                    return int_holidays_dict['is_holiday', date][country]
                except:
                    return get_holiday_status(country, date)

            elif country in exceptions:

                country = exceptions[country]

                try:

                    return int_holidays_dict['is_holiday', date][country]

                except:

                    return get_holiday_status(country, date)

            elif "-" in country.lower():

                country = " ".join(country.lower().split("-"))

                try:

                    return int_holidays_dict['is_holiday', date][country]

                except:

                    return get_holiday_status(country, date)
            else:
                #Need to fix this default option
                #For now, we are returning '0' IF country not in Holidays_DICT
                return 0
    except:
        
        return 0

In [25]:
!ls -alrth /data_data/session_length/session_data*

-rw-r--r-- 1 hduser hadoop 1.6K Sep 16 13:08 /data_data/session_length/session_data_10.csv
-rw-r--r-- 1 hduser hadoop 142K Sep 16 13:33 /data_data/session_length/session_data_1k.csv
-rw-r--r-- 1 hduser hadoop 922K Sep 16 13:47 /data_data/session_length/session_data_10k.csv
-rw-r--r-- 1 hduser hadoop 329M Oct 16 10:38 /data_data/session_length/session_data_1M_is_holiday.tsv
-rw-r--r-- 1 hduser hadoop    0 Oct 19 03:23 /data_data/session_length/session_data_1k_is_holiday.tsv
-rw-r--r-- 1 hduser hadoop    0 Oct 20 13:13 /data_data/session_length/session_data_10_is_holiday.tsv


In [26]:
!head -n 41 /data_data/session_length/session_data_1M.tsv

head: cannot open '/data_data/session_length/session_data_1M.tsv' for reading: No such file or directory


In [28]:
user_sessions_DF = pd.read_csv('/data_data/session_length/session_with_genre_ALL_10_25_2018.csv')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 16, saw 4


In [83]:
with open('session_with_time_of_day_all.csv', 'r') as f:
    lines = f.readlines()[50]
    for line_ in lines:
        line=",".join(line_)
        print(line)

"
u
s
e
r
_
0
0
0
0
0
1
"
	
[
"
'
2
0
0
9
-
0
5
-
0
2
T
1
4
:
3
7
:
1
1
'
"
,
 
"
'
a
3
9
3
4
f
4
7
-
c
4
c
c
-
4
e
7
3
-
8
b
3
7
-
c
e
4
1
7
7
5
7
9
7
d
4
'
"
,
 
"
'
L
i
s
a
 
S
h
a
w
'
"
,
 
"
'
'
"
,
 
"
'
I
n
s
i
d
e
 
M
y
 
L
o
v
e
 
(
E
x
t
e
n
d
e
d
 
M
i
x
)
'
"
,
 
"
'
1
4
'
"
,
 
"
'
m
'
"
,
 
"
'
'
"
,
 
"
'
J
a
p
a
n
'
"
,
 
"
'
A
u
g
 
1
3
,
 
2
0
0
6
'
"
,
 
"
'
2
0
0
6
-
0
8
-
1
3
'
"
,
 
"
<
D
s
t
T
z
I
n
f
o
 
'
A
s
i
a
/
T
o
k
y
o
'
 
L
M
T
+
9
:
1
9
:
0
0
 
S
T
D
>
"
,
 
"
d
a
t
e
t
i
m
e
.
d
a
t
e
t
i
m
e
(
2
0
0
9
,
 
5
,
 
3
,
 
8
,
 
3
7
,
 
1
1
,
 
t
z
i
n
f
o
=
<
D
s
t
T
z
I
n
f
o
 
'
A
s
i
a
/
T
o
k
y
o
'
 
J
S
T
+
9
:
0
0
:
0
0
 
S
T
D
>
)
"
,
 
"
'
M
o
r
n
i
n
g
'
"
,
 
"
'
S
u
n
d
a
y
'
"
]




In [29]:
user_sessions_DF = pd.read_csv('/data_data/session_length/session_with_genre_ALL_10_25_2018_pipe.csv')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 16, saw 4


In [107]:
user_sessions_DF.head()

,userid,gender,age,country,startdate,day_of_week,timeofday,sessionid,session_start,session_length
0,user_000001,m,NaN,Japan,2006-08-13,sunday,noon,1,2006-08-13 13:59:20,230
1,user_000001,m,NaN,Japan,2006-08-15,tuesday,morning,1,2006-08-15 12:27:51,170
2,user_000001,m,NaN,Japan,2006-08-16,wednesday,morning,1,2006-08-16 10:27:16,43
3,user_000001,m,NaN,Japan,2006-08-16,wednesday,noon,2,2006-08-16 13:43:02,86
4,user_000001,m,NaN,Japan,2006-08-17,thursday,noon,1,2006-08-17 13:33:27,115


In [108]:
countries_in_sessions = list([ctry for ctry in sessionsDF.select('country').distinct().toPandas()['country'] if ctry])

In [109]:
countries_in_sessions[:10]

[u'British Indian Ocean Territory',
 u'Congo, the Democratic Republic of the',
 u'Sweden',
 u'Singapore',
 u'Turkey',
 u'Germany',
 u'Northern Mariana Islands',
 u'France',
 u'Greece',
 u'Algeria']

In [110]:
get_holiday_status('united states',date)

1

In [111]:
int_holidays_dict['is_holiday', date][:10]

country
algeria                0
american samoa         1
andorra                1
angola                 1
anguilla               1
antigua and barbuda    1
argentina              1
armenia                0
australia              1
austria                1
Name: (is_holiday, Dec-25-2008), dtype: int64

##### Check that we have holiday data for each country

In [112]:
get_holiday_status_udf = udf(lambda country, date:  int(get_holiday_status(country,date)), IntegerType())

In [113]:
session_with_holiday_DF = sessionsDF.withColumn('is_holiday',get_holiday_status_udf(col('country'),col('session_date')))

In [114]:
session_with_holiday_DF.printSchema()

root
 |-- userid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- artid: string (nullable = true)
 |-- artname_: string (nullable = true)
 |-- traid: string (nullable = true)
 |-- traname_: string (nullable = true)
 |-- session_date: date (nullable = true)
 |-- session_window: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- registered: string (nullable = true)
 |-- reg_date: date (nullable = true)
 |-- is_holiday: integer (nullable = true)



In [115]:
session_with_holiday_DF.show()

+-----------+--------------------+--------------------+------------------+--------------------+--------------------+------------+--------------+------+---+-------------+-----------+----------+----------+
|     userid|           timestamp|               artid|          artname_|               traid|            traname_|session_date|session_window|gender|age|      country| registered|  reg_date|is_holiday|
+-----------+--------------------+--------------------+------------------+--------------------+--------------------+------------+--------------+------+---+-------------+-----------+----------+----------+
|user_000066|2009-03-27T17:18:26Z|                null|   The Amen Corner|                null|Our Love (Is In T...|  2009-03-27|            17|     f| 20|United States|May 9, 2006|2006-05-09|         0|
|user_000066|2009-03-08T04:06:21Z|2ceb4e66-4eaa-4db...|Natalia Lafourcade|a8864539-e4cc-4fc...|          En El 2000|  2009-03-08|             4|     f| 20|United States|May 9, 2006|200

In [116]:
session_with_holiday_DF.sample(0.00001).show(100, False)

+-----------+--------------------+------------------------------------+---------------------------+------------------------------------+---------------------------------------------+------------+--------------+------+----+--------------+------------+----------+----------+
|userid     |timestamp           |artid                               |artname_                   |traid                               |traname_                                     |session_date|session_window|gender|age |country       |registered  |reg_date  |is_holiday|
+-----------+--------------------+------------------------------------+---------------------------+------------------------------------+---------------------------------------------+------------+--------------+------+----+--------------+------------+----------+----------+
|user_000577|2008-03-29T23:12:34Z|9c9f1380-2516-4fc9-a3e6-f9f61941d090|Muse                       |989149e3-dfd5-4187-9d4a-aa72fe82dc1f|Feeling Good                                 

### Save as file (via toPandas())

In [117]:
#spark.conf.set("spark.sql.execution.arrow.enabled", "false")
#session_with_holiday_DF.toPandas().to_csv("session_with_holiday_all.csv", sep='\t', encoding='utf-8',index=False)
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [118]:
!tail -n 200 session_with_holiday_all.csv 

user_001000	m		United States	2009-02-05	thursday 	night	1	2009-02-05	56	0
user_001000	m		United States	2009-02-05	thursday 	morning	2	2009-02-05	49	0
user_001000	m		United States	2009-02-05	thursday 	evening	3	2009-02-05	24	0
user_001000	m		United States	2009-02-06	friday   	night	1	2009-02-06	302	0
user_001000	m		United States	2009-02-06	friday   	noon	2	2009-02-06	55	0
user_001000	m		United States	2009-02-07	saturday 	night	1	2009-02-07	3	0
user_001000	m		United States	2009-02-07	saturday 	night	2	2009-02-07	38	0
user_001000	m		United States	2009-02-07	saturday 	night	3	2009-02-07	50	0
user_001000	m		United States	2009-02-07	saturday 	noon	4	2009-02-07	281	0
user_001000	m		United States	2009-02-08	sunday   	evening	1	2009-02-08	252	0
user_001000	m		United States	2009-02-09	monday   	night	1	2009-02-09	47	0
user_001000	m		United States	2009-02-09	monday   	night	2	2009-02-09	122	0
user_001000	m		United States	2009-02-09	monday   	evening	3	2009-02-09	34	0
user_001000	m		United States	

### Compute Delta since last song

In [62]:
!head session_data

userid	gender	age	country	startdate	day_of_week	timeofday	sessionid	session_start	session_length	is_holiday
user_000001	m		Japan	2006-08-13	sunday   	noon	1	2006-08-13	230	0
user_000001	m		Japan	2006-08-15	tuesday  	morning	1	2006-08-15	170	0
user_000001	m		Japan	2006-08-16	wednesday	morning	1	2006-08-16	43	0
user_000001	m		Japan	2006-08-16	wednesday	noon	2	2006-08-16	86	0
user_000001	m		Japan	2006-08-17	thursday 	noon	1	2006-08-17	115	0
user_000001	m		Japan	2006-08-18	friday   	evening	1	2006-08-18	46	0
user_000001	m		Japan	2006-08-19	saturday 	morning	1	2006-08-19	113	0
user_000001	m		Japan	2006-08-20	sunday   	noon	1	2006-08-20	112	0
user_000001	m		Japan	2006-08-21	monday   	night	1	2006-08-21	18	0
